In [ ]:
# ==== MONTAJE Y RUTAS ====
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

RAIZ = "/content/drive/My Drive/videoscombi"
RUTA_VALIDOS    = f"{RAIZ}/videosvalidos"
RUTA_NOVALIDOS  = f"{RAIZ}/videosnovalidos"
MODELO_SALIDA   = "/content/drive/My Drive/modelo_saltos_novalidos_seq.h5"

# ==== UTILS ====
import os, re, cv2, random, numpy as np, tensorflow as tf
random.seed(42); np.random.seed(42); tf.random.set_seed(42) #para hacerlo determinante se fijan las semillas hasta los shuffle.

#parametros base de la red neuronal, para determinar como se van a procesar los datos.

IMG_SIZE          = (160, 160) #tamaño al que se redimensiona cada frame.
FRAMES_POR_VIDEO  = 24 #cuantos frames equiespaciados representan el salto
BATCH_SIZE        = 8 # lote de entrenamiento
VAL_SPLIT         = 0.2 # fracción de validación.

def natkey(s): return [int(t) if t.isdigit() else t.lower() for t in re.split(r"(\d+)", str(s))] #para tener orden natural, digit si son numeros ok, split para hacerlos int.
def es_imagen(p): return str(p).lower().endswith((".jpg",".jpeg",".png")) #reconoce extensiones de imagenes.

def listar_subcarpetas(carpeta): #recorre subcarpetas porque son un salto individual
    try:
        subs = [os.path.join(carpeta,d) for d in os.listdir(carpeta) #Devuelve todos los nombres de archivos y carpetas dentro de "carpeta"
                if os.path.isdir(os.path.join(carpeta,d))] #pura subcarpeta y join para unir carpeta con subcarpeta y tener la ruta.
    except FileNotFoundError: #regresa una lista vacia en lugar de dar error.
        return []
    return sorted(subs, key=natkey) #ordena las subcarpetas con natkey.

def listar_frames_en_carpeta(carpeta): #lista las imagenes que sirven en una carpeta e ignora archivos que no sean imagenes y overlays.
    if not os.path.isdir(carpeta): return [] #si no exiaste la carpeta, regresa una lista vacia
    fs = []
    for f in os.listdir(carpeta): #recorre todos los nombres dentro de la carpeta.
        if not es_imagen(f): continue #asegura que sean imagenes.
        # ignorar frames con overlays/resultados.
        low = f.lower()
        if ("resultado" in low) or ("_result" in low) or ("overlay" in low):
            continue
        p = os.path.join(carpeta, f) #ruta del archivo
        if os.path.isfile(p):
            fs.append(p) #solo añade si realemnte hay archivo.
    return sorted(fs, key=natkey) #devuelve la lista de rutas de imágenes, ordenadas naturalmente con el natkey.

#sirve para resumir cada salto completo en una muestra representativa y genera una nueva lista con los frames seleccionados según los índices.

def muestrear_k(items, k): #rutas de frames y elementos que se quieren obtener de forma equidistante
    if not items: return [] # para verificar que item no este vacio, si lo esta regresa lista vacia.
    if len(items) <= k: # si imtem ≤ k repite elementos evita errores.
        idx = np.linspace(0, len(items)-1, num=k, dtype=int) #0 primer indice, len es el último válido, numk porque k posiciones, int para enteros, porque el lins hace floats
        return [items[i] for i in idx] # de items recorre todo idx, construye una nueva lista y la devuelve.


#Esta función prepara las imagenes, convierte los archivos de imagen en tensores numéricos normalizados en el formato y rango que MobileNetV2 necesita.

def leer_redimensionar(fp): #fp es la ruta del archivo
    im = cv2.imread(fp) #con open cv lee imagenes y regresa un array de numpy con valores de cada pixel
    if im is None: return None #si no se puede leer la imagen, regresa none, evita errores.
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) #open cv solo lee rgb por eso se pasa de bgr y tambien TF/K.
    im = cv2.resize(im, IMG_SIZE) #redimensiona la imagen al de IMG_SIZE, para que todas las fotos tengan mismo tamaño, es requisito.
    # Preprocess de MobileNetV2
    im = tf.keras.applications.mobilenet_v2.preprocess_input(im.astype(np.float32)) #pasa las imagenes a float32 por TF luego MNV2, escala pixeles desde (0 a 255) y el rabgo de -1 a 1 porque MNV2 lo necesita asi
    return im # devuelve la imagen en RGB tamaño (160, 160) y normalizada para MobileNetV2.

#organiza el dataset agrupando los frames pertenecientes a cada salto dentro de su clase correspondiente (“válido” o “no válido”).

def recolectar_grupos_de_clase(ruta_clase, etiqueta): #etiqueta es número de clase asociado 0 para válidos 1 para noválidos.
    grupos = [] #aca se guardan los conjuntos de frames encontrados.
    if not os.path.isdir(ruta_clase): return grupos #verifica si la ruta existe y es carpeta si no, regresa lista vacia.
    # subcarpetas
    for sub in listar_subcarpetas(ruta_clase): #recorre las subcarpetas de ruta_clase y cada una representa un salto.
        fs = listar_frames_en_carpeta(sub) #funcion auxiliar que devuelve las imagenes dentro de la subcarpeta con fs una lista de rutas de imágen.
        if fs: grupos.append((fs, etiqueta, sub)) # si fs tiene frames, crea una tupla al listado de grupos, lista frames, numero de clase (0,1) y donde estan esos frames.
    # frames directo en la raíz
    fs_root = listar_frames_en_carpeta(ruta_clase) # y subcarpetas igual.por si acaso para que funcione aunque no esten en subcarpetas (si lo estan, es para hacer el trabajo más general)
    if fs_root: grupos.append((fs_root, etiqueta, ruta_clase)) #si en la carpeta hay frames los agrega como un grupo más, en el mismo formato de tupla.
    return grupos # devuelve la lista de grupos encontrados y cada elemento de "grupos" tiene el mismo formato de tupla. Lista de frames (fs) Etiqueta de clase (0 o 1) Ruta de origen (sub o ruta_clase)

# ==== CHEQUEO DURO DEL DATASET ====
gr_validos   = recolectar_grupos_de_clase(RUTA_VALIDOS, 0) #llama a la función con la carpeta de validos (0), regresa una lista de frames para los asosciados a 0.
gr_novalidos = recolectar_grupos_de_clase(RUTA_NOVALIDOS, 1) #hace lo mismo pero para inválidos.
print(f"[INFO] grupos válidos: {len(gr_validos)}  |  no válidos: {len(gr_novalidos)}") #muestra cuantos se encuentran en cada clase y cuántos grupos (subcarpetas o conjuntos de frames) se encontraron en la carpeta de válidos, lo mismo para inválidos.

# Muestra 3 grupos por clase con #frames (asegura que NO aparezca 0)
for titulo, grupos in [("VALIDOS", gr_validos), ("NO VALIDOS", gr_novalidos)]: #itera string que son titulos y las tuplas los grupos.
    print(f"\n{titulo}:") #imprime los nombres
    for fs, y, path in grupos[:3]: #este segudo bluce es para verificar, recorre los 3 primeros elementos de la lista de grupos
        print(f"  {os.path.basename(path)} -> {len(fs)} frames (etiqueta={y})") #verifica si hay frames en cada grupo, la etiqueta esta bien y si los grupos estan recolentados.

# Chequeo crítico: aborta si no hay datos
assert len(gr_validos) + len(gr_novalidos) > 0, "No encontré grupos con frames. Verifica rutas." #suma los lens y esa suma > 0 asi se ve si hay grupo con frames.

Mounted at /content/drive
[INFO] grupos válidos: 83  |  no válidos: 19

VALIDOS:
  Men's Long Jump Final _ Tokyo Replays - YouTube - Google Chrome 2025-01-03 15-48-46 -> 258 frames (etiqueta=0)
  Men's Long Jump Final _ Tokyo Replays - YouTube - Google Chrome 2025-01-03 15-48-46_result -> 258 frames (etiqueta=0)
  Men's Long Jump Final _ Tokyo Replays - YouTube - Google Chrome 2025-01-03 15-49-07 -> 202 frames (etiqueta=0)

NO VALIDOS:
  vn1 -> 189 frames (etiqueta=1)
  vn2 -> 253 frames (etiqueta=1)
  vn3 -> 229 frames (etiqueta=1)


In [ ]:
# =========================================================
#  ENTRENAR CLASIFICADOR "NO VÁLIDO" (secuencias de frames)
#  TODO-EN-UNO  — versión con correcciones A–E
#    (A) preprocess_input de MobileNetV2
#    (B) orden natural y filtro de overlays
#    (C) gen_mini sin doble lectura
#    (D) class_weight simétrico (inverso a la frecuencia)
#    (E) callbacks: ModelCheckpoint + EarlyStopping
# =========================================================
import os, re, cv2, random, numpy as np, tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# ----------- CONFIG -----------
RAIZ = "/content/drive/My Drive/videoscombi" #aca estan las 2 subcarpetas, válidos y no válidos
RUTA_VALIDOS   = os.path.join(RAIZ, "videosvalidos")
RUTA_NOVALIDOS = os.path.join(RAIZ, "videosnovalidos") #con join para que la unión funcione bien en todo sistema operativo.

IMG_SIZE = (160, 160)        # tamaña al que se redimenciona cada frame (puedes subir a (192,192) si tienes GPU rápida)
FRAMES_POR_VIDEO = 24        # el salto se reduce a 24 frames (baja a 12 si quieres velocidad o sube si tienes GPU rápida)
BATCH_SIZE = 4               # lote por entrenamiento, en cada paso de entrenamiento hay 4 secuencias de video (baja a 2 si la RAM es poca pero tardará más)
VAL_SPLIT = 0.2              # fracción de datos usados para validación el otro .8 es para entrenar.
EPOCHS    = 6                # las veces que el modelo verá todo el dataset mientras entrena (baja para más velocidad o sube para mejorar resultados con precaución al sobreentrenamiento.)
SEED      = 42               # valor clasico para la semilla de aleatoriedad es para que los resultados sean reproducibles en todo, o sea muestro y shuffle.
MODELO_SALIDA = "/content/drive/My Drive/modelo_saltos_novalidos_seq.h5" #guardo en .h5 para que se guarde sin reentrenar, ahorra tiempos.

random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED) #fijo la semilla de aleatoriedad a 3 librerias la estandar de py, operaciones vectoriales y para inicialización de pesos y mezclado de datos es importante para depurar.

# ----------- utilidades de IO -----------
def listar_subcarpetas(p): #recorre el directorio p y devuelve las subcarpeta que tiene.
    try:
        return sorted([os.path.join(p,d) for d in os.listdir(p) if os.path.isdir(os.path.join(p,d))]) #sorted para que este ordenado alfabéticamente mas no natural, eso es de natkey.
    except FileNotFoundError: # regresa una lista vacia para no tirar error.
        return [] #es para encontrar todas las carpetas con los saltos.

def natkey(s):  # (B) orden natural para respetar secuencia temporal
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r"(\d+)", str(s))] #split para separar la cadena s en su parte númerica y de texto.
                                                                                      #convierte las partes númericas a int y el txt a minusculas asi ya tienen un orden natural de esta forma "frame1.jpg"
def listar_frames_en_carpeta(p): #recorre p y recoge puras fotos.
    # (B) filtro de overlays/resultados y orden natural
    exts = (".jpg",".jpeg",".png") #a esto me refiero con puras fotos.
    ban  = ("resultado", "_result", "overlay") ]#filtra este tipo de nombres para evitar imagenes marcadas o generadas de intentos anteriores.
    try:
        fs = [] #se inicia una lista vacia llamada fs y ahi se va a guardar las rutas completas de imagenes válidas que estan en p.
        for f in os.listdir(p): #nombres de archivos y carpetas dentro de p. f es la variable no ruta de los elementos tipo frame 1.jpg, overlay_frame1.png, etc.
            low = f.lower() #pasa el nombre a minuscula para que las comprobaciones ean más sencillas.
            if not low.endswith(exts): continue # si no termina con nombre de foto se la salta (exts = (".jpg",".jpeg",".png"))
            if any(b in low for b in ban): continue #si el nombre del archivo tiene la palabra, resultado, _result, overlay, se salta.
            fs.append(os.path.join(p,f)) #si pasa los filtros pasa a fs y os.path.join (p,f)construye la ruta.
        return sorted(fs, key=natkey) #ordena los archivos por número de frame. #cuando se revisan todos los datos se regresa fs pero ordenada en buen orden, o sea que frame 1, frame 2 y no frame 1, frame 10 como seria alfabeticamente.
    except FileNotFoundError: #si p no existe regresa lista vacia, para que no se detenga el programa.
        return []

def armar_grupos(ruta, etiqueta): #define la funcion que construye grupos, con ruta un str (videos válidos) y etiqueta un int (0)
    grupos = [] #cada elemnto es una tupla de la forma :(fs, etiqueta, ruta_grupo)
    for sub in listar_subcarpetas(ruta): #revisa subcarpetas de ruta, porque cada subcarpeta es un video
        fs = listar_frames_en_carpeta(sub) #todas las imagenes validas dentro de sub, filtra por extensión, excluye "resultado, _result, overlay" y ordena de forma natural.
        if fs: #para robustez, agrega al grupo si hay al menos un frame, esto evita guardar vacios.
            grupos.append((fs, etiqueta, sub)) #agrega a un nuevo grupo que tiene fs validos, etiquetas constantes y el sub como ruta para saber de donde salen los grupos.
    fs_root = listar_frames_en_carpeta(ruta)  # este bloque es por si no hya subcarpetas y los frames estan en raíz, fs_root: lista de frames válidos directamente en la raíz de la clase.
    if fs_root: # si no esta vacia, se añade un nuevo grupo con su "ruta_grupo" como la propia ruta, asi se tiene cubierto.
        grupos.append((fs_root, etiqueta, ruta))
    return grupos #devuelve la lista complta de grupos con buen fs, etiqueta y ruta.

gr_validos   = armar_grupos(RUTA_VALIDOS,   0) #se llama a la función armar_grupos y esta es la ruta del disco con videos válidos.
gr_novalidos = armar_grupos(RUTA_NOVALIDOS, 1) #lo mismo pero para no válidos. y las 2 son tuplas.

print(f"[INFO] grupos válidos: {len(gr_validos)} | no válidos: {len(gr_novalidos)}") #para verificar si la carga del dataset es correcta y cuantos grupos de válidos y no se encuentran, no frames, si subcarpetas.
if (len(gr_validos)+len(gr_novalidos)) == 0: #para no seguir entrenando si no hay datos, como antes usma los 2 grupos y si es 0, no encontro ningún grupo.
    raise RuntimeError("No hay frames en las carpetas. Verifica RAIZ y que existan .jpg/.png.") #si es el caso lanza este mensaje en consola.

# ----------- muestreo y preprocesado -----------
def muestrear_k(fs, k): #para seleccionar k frames de una lista de ruta de frames, fs es la lista y k los k frames equiespaciados.
    if not fs: return []#si fs esta vacio, devuelve una lista vacía.
    if len(fs) <= k: #si la cantidad de frames es ≤ k hay un problema.
        return (fs + [fs[-1]]*(k-len(fs)))[:k] #pero se soluciona asi. agarra la losta fs completa y se añaden copias del ultimo frame hasta llegar a k.
    idx = np.linspace(0, len(fs)-1, num=k, dtype=int) #el linspace es para equispaciar y si hay más agarra entre 0 y último, o sea: len(fs)=100, k=10 idx = [0, 11, 22, 33, 44, 55, 66, 77, 88, 99]
    return [fs[i] for i in idx] #devuelve la lista calculada con idx fs[i] es una ruta a un archivo de imagen, esto nos da las k rutas de frames equispaciadas uniformemente.

#este es una función que pasa archivos del disco a un tensor listo para pasar por MobileNetv2

def leer_redimensionar(fp, size=IMG_SIZE): #ruta del archivo de imagen y el tamaño al que se redimensiona.
    #la función oficial de MobileNetv2 para no hacer un [0,1] dividiendo por 255 preprocess_input de MobileNetV2 ([-1,1]) es la arquitectura que necesita porque se entreno con esa normalización.
    img = cv2.imread(fp) #usa opencv para leer la imagen de la ruta fp el resultado es un array de numpy de la forma: (alto, ancho, 3)
    if img is None: return None #si la lectura falla por le motivo que sea regresa un none. ayuda a que el pipeline vea si es none para saltarla.
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #pasa de BRG a RGB impprtante porque el opencv esta en bgr y tensor y keras necesita rgb.
    img = cv2.resize(img, size, interpolation=cv2.INTER_AREA) # redimensiona al tamaña 160, 160. inter_area es para reducior imagenes manteniendo calidad y evita aliasing, ya con la misma forma en imagenes se puede entrenar.
    img = img.astype(np.float32) #pasa el array a float32 porque asi lo trabaja tensorflow no float64 y el preprocess espera floats no enteros de 0-255.

    return preprocess_input(img) # fnción de MobileNetv2 es de [-1 a 1] o sea si tenemos pixel 0 seria -1.0, Pixel 127 → ~ -0.0039 Pixel 255 → +1.0

# ----------- split estratificado ----------- # es la que se encarga de dividir el data set en entrenamiento y validación.
def split_estratificado(V, NV, val_split=0.2, seed=SEED): #validos, no, validación, para que la división sea reproducible.
    random.seed(seed) #fijo la semilla para que sea reproducible.
    V = V.copy(); NV = NV.copy() # hace copias de las listas originales para no modificarlas, shuffle no altera las listas externas que se usen más adelante.
    random.shuffle(V); random.shuffle(NV) #mezcla los elementos de manera aleatoria, cada clase por separado que da estratificación.
    nV_val  = max(1, int(len(V)*val_split)) if len(V)>0 else 0 # número proporcional de videos válidos para validación. el if es que si la lista esta vacia regrese 0 de una. y el val es para ver cuantos elementos se van a validación dependiendo la proporción.
    nNV_val = max(1, int(len(NV)*val_split)) if len(NV)>0 else 0 #lo mismo pero para no válidos. int redondea enteros para abajo, max es para asegurar que hay elementos y al menos uno pase a validación.
    items_val = V[:nV_val] + NV[:nNV_val] # los primeros nV y nNV se van a validación.
    items_tr  = V[nV_val:] + NV[nNV_val:] # los demas se quedan en entrenamiento.
    random.shuffle(items_val); random.shuffle(items_tr) #mezcla conjunto de validación y de entrene para que no queden agrupados en clase, o sea yodos los válidos primero, rompe el orden inicial, pero sigue en balance.
    return items_tr, items_val #regresa grupos de entrenamiento y grupos de validación. (se sigue conservando el mismo formato listaframes, etiqueta y ruta)

items_tr, items_val = split_estratificado(gr_validos, gr_novalidos, VAL_SPLIT) #llama al split y válido tiene forma de (frames, 0, ruta) y noválido (frames, 1, ruta) y el val pues el .2.
print("Train V/NV:", sum(1 for _,y,_ in items_tr if y==0), sum(1 for _,y,_ in items_tr if y==1)) # se recorre cada tupla(fs,label,ruta) _ ignora, y label, if 0 para válido y el sum es para sumar elementos que cumplen la misma condición.
print("Val   V/NV:", sum(1 for _,y,_ in items_val if y==0), sum(1 for _,y,_ in items_val if y==1)) #lo mismo pero para validación, con el objetico de que es balanceable los V y NV en train y val.

# ----------- generadores + tf.data -----------
def gen(items_lista, k=FRAMES_POR_VIDEO): #reproduce en formato stream, muy importante porque uso yield, tambien porque asi funciona mejor TensorFlow, no uses return.
    for fs, y, _ in items_lista: #itera sobre cada grupo y desempaqueta los usa el fs, y, ruta. (no se usan)
        sel = muestrear_k(fs, k) #selecciona k rutas de fs (lista de rutas de imagenes equispaciadas) Si len(fs) > k: muestreo uniforme con np.linspace. si len(fs) <= k: rellena repitiendo el último frame hasta llegar a k, asi se garantiza que todas tengan los 24.
        batch = [] #acumula los k frames ya leídos y procesados (no solo las rutas)
        for fp in sel: #
            arr = leer_redimensionar(fp) #lee la imagen y pasa de bgr a rgb, redimenciona a (160 x 160), pasa a float 32, normaliza con preprocess_input de MNV2 de [-1, 1]
            if arr is not None: si falla se omite
                batch.append(arr) #si sale bien se añade el tensor de la imagen a batch. aqui el batch tiene tensores de entre 0 y k y de forma (H, W, 3)
        if not batch: # si no se puede leer ningun frame, salta el grupo y pasa al siguiente. eso para no romper el np.stack con ejemplos vacíos.
            continue
        while len(batch) < k:  # rellena si faltó alguno
            batch.append(batch[-1]) # con el último frame válido.
        x = np.stack(batch, axis=0)   #aplica la lista batch en un solo array 4D de forma (K,H,W,3) (imagenes, largo, ancho, color)
        yield x.astype(np.float32), np.int32(y) #x es la frecuencia de k frames ya normalizados y float 32, y es etiqueta ya en int32 y como es un gen carga todo 1 a 1, no todos, ideal porque usamos tf.data.Dataset.from_generator.

spec_x = tf.TensorSpec(shape=(FRAMES_POR_VIDEO, IMG_SIZE[0], IMG_SIZE[1], 3), dtype=tf.float32) # el spec es para decirle a TF que forma tienen mis tensores y sus datos (24,160,160,3)
spec_y = tf.TensorSpec(shape=(), dtype=tf.int32) # shape para tensor escalar, no tiene dimensiones, solo 1 valor, o sea solo un número no la lista, y ese int porque es compatible con las etiquetas (0 y 1)



#se crea un dataset con generator que ve, 1 a 1 los (x,y) que el gen item va yieldando, lamba es para fijar item del DS asi tensorflow lo puede recrear cuando haga falta.
#output describe forma y tipo de lo que emite el gen para X el tensor (k,h,w,3) con float32 para Y el de () con int32
#shuffle barajea con un buffer de 256 y seed lo hace reproducible, batch pasa x a (BATCH_SIZE,K,H,W,3) y con Y (BATCH_SIZE,)
#prefetch hace un pipeline cuando la GPU entrene con batch n la cpu prepara el batch n+1. y el autotune es para que tf elija el tamaño del  prefetch según el hardware.
ds_tr  = tf.data.Dataset.from_generator(lambda: gen(items_tr),  output_signature=(spec_x, spec_y)) \
                        .shuffle(256, seed=SEED).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_val = tf.data.Dataset.from_generator(lambda: gen(items_val), output_signature=(spec_x, spec_y)) \ #lo mismo sin shuffle pero si con batch + prefetch, para rendimiento en la validación.
                        .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(f"[INFO] train videos: {len(items_tr)} | val videos: {len(items_val)}") #cuántos saltos quedaron en que grupo despues del split que es la separación de mi dataset en entrenamiento y validación y es para entrenar y evaluar. para que no memorice la red.

# ----------- modelo -----------
#la función que construye el modelo de deep learning.
def construir_modelo(img_size=IMG_SIZE, k=FRAMES_POR_VIDEO, base_trainable=False, lr=1e-3): #(si es true entrena los pesos de MNV2 si es false se queda congelado, learning rate, que tan rapido aprende el optimizador adam)
    inp = layers.Input(shape=(k, img_size[0], img_size[1], 3)) #capa de entrada (k, alto, ancho. color) (24,160,160,3)
    base = tf.keras.applications.MobileNetV2(include_top=False, input_shape=img_size+(3,), weights="imagenet") #(no incluye la ultima capa de clasificación de IN, cada imagen es (160,160,3), incia con pesos preentrenados)
    base.trainable = base_trainable # controla si MobileNet entrena o no, false se congela y no modifica sus pesos, true, modifica algunos pesos bueno para fine tuning porque tengo muchos datos, ya tendriamos un mapa de características cnn 2D.
    x = layers.TimeDistributed(base)(inp) #aplica MobileNet independientemente a cada frame,m decirle a keras pon MN  cada frame indepedientemente usando los mismos pesos.
    x = layers.TimeDistributed(layers.GlobalAveragePooling2D())(x) # arriba (batch, k, 5, 5, 1280) ahora (batch, k, 1280) promedia en el espacio.


    x = layers.GlobalAveragePooling1D()(x) #promedia en el tiempo, haciedo ya un vector de 1D (batch, k, 1280) con k el tiempo.

    x = layers.Dropout(0.3)(x) #apaga aleatoriamente el 30% de valores para que la red sea más independiente,(batch, 1280) sin k porque ya no es frame a frame es el resumen de un video.
    out = layers.Dense(1, activation="sigmoid")(x) # prob. de inválido, multiplica por una matriz (1280,1) y suma sesgo, y por el activador es de 0 a 1. de .5 para arriba es mas prob que sea inválido.
    model = models.Model(inp, out) #conecta entradas(inp) con salidas(out) por tf.keras.Model representa toda la red, ya esta lista, para predecir y entrenar.
    model.compile( #para decirle a keras como entrenar el modelo.
        optimizer=tf.keras.optimizers.Adam(lr), # adam con el 1e-3=.001 es mi favorito.
        loss="binary_crossentropy", # es válido o inválido, 1 o 0, por eso uso esa función de pérdida.
        metrics=["accuracy", tf.keras.metrics.AUC(name="auc")] #accuracyes el % de predicción correcta , calcula area bajo la curva AUC-ROC (que tan bien distingue entre valores positivos y negativos)
    ) #cierra model.compile
    return model #ya esta el modelo (Input → MobileNetV2 → GlobalPooling → Dense(sigmoid))

model = construir_modelo(base_trainable=False, lr=1e-3) #llamo a construir_modelo(...), base_trainable... se usa para congelar caracteristicas por el MNV2, lo que resulte se queda en model, es para guardar basicamente.
model.summary() #resumen tabular cada capa con su nombre

#Input: una secuencia de 24 frames de tamaño 160×160×3.
#MobileNetV2 (congelada): extrae mapas de características (5×5×1280).
#GlobalAveragePooling2D: convierte cada frame en un vector de 1280.
#GlobalAveragePooling1D: condensa la secuencia en un solo vector de 1280.
#Dropout: regulariza.
#Dense(1, sigmoid): devuelve una probabilidad de “inválido”.




# ----------- (Opcional) overfit mini — sanity check -----------
#para sabr si el pipeline de datos funciona, ver si la red puede aprender
mini = gr_validos[:2] + gr_novalidos[:2] #toma los 2 primeros grupos de V y NV
def gen_mini(): #usa yield para hacer pares (x,y) para el subset de mini
    # (C) una sola lectura por frame
    for fs,y,_ in mini: #cada grupo de estso es una tupla (fs, y, ruta)
        sel = muestrear_k(fs, FRAMES_POR_VIDEO) # selecdciona k frames equispaciados de fs y si hay menos repite.
        batch = [] #recorre sel, lee cada imagen y preprocesa con leer_redimensionar.cv2.imread de bgr→rgb→img_size→float32→preprocess_input(MobileNetV2: rango [-1,1])
        for fp in sel:
            arr = leer_redimensionar(fp)
            if arr is not None: #si un frame falla, lo omite
                batch.append(arr)
        if not batch: continue #si ningun frame se puede leer, salta el grupo
        while len(batch) < FRAMES_POR_VIDEO: batch.append(batch[-1]) # si faltan frames rellena con el penúltimo hasta tener los 24.
        x = np.stack(batch, axis=0) # aplica los k tensores de forma (H,W,3) → un tensor 4D (K, H, W, 3) para un video.
        yield x.astype(np.float32), np.int32(y) #saca el par de x con (K,H,W,3) float32 y Y con escalar int32

if len(mini) > 0: # solo ejecuta el test si al menos hay un grupo en mini.
    ds_mini = tf.data.Dataset.from_generator(gen_mini, output_signature=(spec_x, spec_y)).batch(2) #crea Dataset de TF para genmini, describe forma y tipo, bnatch agrupa en lotes de 2 secuencias.
    print("\n[TEST] Overfit rápido a 4 videos…") #para distinguir en logs (como tipo stream)
    model_mini = construir_modelo(base_trainable=True, lr=1e-3) # aqui si uso true, para ver que pasa, ver si memoriza se que para pocos datos es false y para muchos true, solo es una prueba.
    hist_mini = model_mini.fit(ds_mini, epochs=4, verbose=0) # 4 epocas solo para ver si sube y aprende.verbose=0 es porque lo quiero logs, solo es chequeo rapido.
    print("acc mini:", float(hist_mini.history["accuracy"][-1]), "auc mini:", float(hist_mini.history["auc"][-1])) #ver como le fue en accuracy y auc.
    #ambos valores suben bien, porque el modelo tiene capacidad de memorizar 2–4 ejemplos, esoe s bueno.

# ----------- class_weight simétrico (D) -----------
#lo primero es para ver cúantos ejemplos de cada clase hay en el dataset de entrenamiento item_tr es una tupla de creada en split estratificado.
n0 = sum(1 for _,y,_ in items_tr if y==0) #número de grupos de clase 0 válidos
n1 = sum(1 for _,y,_ in items_tr if y==1) # 1 np válidos.
n  = n0 + n1 #número total de grupos de entrenamiento, los sumo.
cw = None #inicia (class_weight) como none, por si alguna clase no tuviera datos en ese escenario no se aplican pesos.
if n0 > 0 and n1 > 0: #solo calcula pesos si las 2 clases existen.
    cw = {0: n/(2*max(1,n0)), 1: n/(2*max(1,n1))} #el factor 2 es porque hay 2 clases el max(1, n0 y n1) son para no dividir sobre 0.
    #Así se asegura que cada clase aporte el mismo peso total al entrenamiento, aunque estén desbalanceadas.

# ----------- callbacks (E) -----------
#
cb = [ #creo la lista cb que tiene modelcheckpoint y earlystopping, keras sabra que tiene que usar estos callbacks cuando use model.fit (entrenamiento)
  tf.keras.callbacks.ModelCheckpoint( #es la clase que crea el callback y guarda cuando la metrica de validación mejora
      MODELO_SALIDA, monitor="val_auc", save_best_only=True, mode="max"), #nombre,CB va a mirar datos de validación (que tan bien separa), solo se guardara cuando mejore el val_auc, mientras más alto auc mejor modelo .
  tf.keras.callbacks.EarlyStopping( # crea el CB evita que siga entrenando de más y sobreajuste y asegura los mejores pesos posibles.
      monitor="val_auc", mode="max", patience=3, restore_best_weights=True) #lo mismo, mas auc mejor, después de 3 épocas si no mejora detienen,
] #importante que cuando para no se queda con los pesos de la última época (puede que no sea buena) restaura a pesos del modelo de la época edl mejor AUC.

# ----------- entrenamiento real -----------
#model fit hace que keras empiece a iterar, calcular pérdidas, actualizar pesos y validar las epocas.
#ds_tr es el data set de entrenamiento ya creado, tiene pares (x,y) con x (4,24,160,160,3) y xon Y la etiqueta escalar 0 o 1
#La DataSet de validación es para medir el rendimiento de las epochs, ve si el modelo sobreajusta los val_loss, val_accuracy, val_auc salen de aquí.
#epochs es rl número de veces que el modelo verá todo el dataset de entrenamiento. (gracias al earlystopping, el entrenamiento puede terminar antes si no mejora el AUC.)
#Class_weigth=cw se aplican los pesos que ya calculamos (cw = {0: w0, 1: w1} Durante el cálculo de pérdida (binary_crossentropy), cada muestra se multiplica por su peso.
#util porque asi se equilibran datasets desbalanceados (los errores en la clase minoritaria cuentan más)
#callbacks=cb son los CB de antes, MC: guarda el mejor modelo según val_auc y ES: detiene entrenamiento si no mejora AUC en 3 epochs seguidos, y restaura el mejor.
#El resultado se guarda en history history.history tiene todos los logs de entrenamiento por época
#loss: pérdida en entrenamiento por epoch. accuracy: precisión en entrenamiento. auc: AUC en entrenamiento. val_loss, val_accuracy, val_auc: las mismas pero en validación.
history = model.fit(ds_tr, validation_data=ds_val, epochs=EPOCHS, class_weight=cw, callbacks=cb)


#mensaje en consola para saber que se guardo bien. Nota si se entrenan 10 épocas, pero fue la 7 le mejor,se guarda la epoch 7.)
model.save(MODELO_SALIDA)
print(f"[OK] Modelo guardado en: {MODELO_SALIDA}")

# ----------- (Opcional) Fine-tuning breve -----------
# Descomenta para afinar últimas capas:
# model.get_layer(index=1).trainable = True  # 'base' es el segundo layer en este grafo
# for l in model.get_layer(index=1).layers[:-5]:  # congela todo menos ~5 capas finales
#     l.trainable = False
# model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
#               loss="binary_crossentropy",
#               metrics=["accuracy", tf.keras.metrics.AUC(name="auc")])
# history_ft = model.fit(ds_tr, validation_data=ds_val, epochs=3, class_weight=cw, callbacks=cb)
# model.save(MODELO_SALIDA)
# print("[OK] Modelo fine-tuned guardado.")


[INFO] grupos válidos: 83 | no válidos: 19
Train V/NV: 67 16
Val   V/NV: 16 3
[INFO] train videos: 83 | val videos: 19
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 24, 160, 160,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 24, 5, 5, 1280) │     2,257,984 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 24, 1280)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

...

21/21 ━━━━━━━━━━━━━━━━━━━━ 138s 3s/step - accuracy: 0.7143 - auc: 0.7516 - loss: 0.6078 - val_accuracy: 0.6842 - val_auc: 0.6875 - val_loss: 0.5113
Epoch 4/6
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7255 - auc: 0.7266 - loss: 0.6301

21/21 ━━━━━━━━━━━━━━━━━━━━ 110s 3s/step - accuracy: 0.7259 - auc: 0.7286 - loss: 0.6279 - val_accuracy: 0.5789 - val_auc: 0.7188 - val_loss: 0.6583
Epoch 5/6
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6466 - auc: 0.7105 - loss: 0.6049

21/21 ━━━━━━━━━━━━━━━━━━━━ 154s 3s/step - accuracy: 0.6517 - auc: 0.7163 - loss: 0.6001 - val_accuracy: 0.7368 - val_auc: 0.7917 - val_loss: 0.4778
Epoch 6/6
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7511 - auc: 0.7664 - loss: 0.5005

21/21 ━━━━━━━━━━━━━━━━━━━━ 116s 3s/step - accuracy: 0.7492 - auc: 0.7666 - loss: 0.5043 - val_accuracy: 0.5789 - val_auc: 0.8125 - val_loss: 0.7526


[OK] Modelo guardado en: /content/drive/My Drive/modelo_saltos_novalidos_seq.h5


In [ ]:
# =========================================================
# Celda A — CONFIG + HELPERS
# =========================================================
import os, re, cv2, random, numpy as np, tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
#os es para navegar en carpetas sin os no se podria listar frames desde las carpetas, usos: (os.listdir, os.path.join, os.path.isdir, os.path.isfile.)
#re es para que los frames se lean en el orden temporal correcto, se usa en natkey(s)
#usos de cv2: cv2.imread: leer imágenes. cv2.cvtColor: cambiar color BGR→RGB. cv2.resize: redimensionar a (160,160), sirve para preparar los frames antes de pasarlos al modelo.
#usos de random: random.seed(SEED): fijar semilla → reproducibilidad. random.shuffle: mezclar listas (split_estratificado). (evita que haya resultados diferentes en cada ejecución)
#numpy: np.linspace:equiespaciar frames. np.stack:imágenes en tensores. np.float32:datos TensorFlow. np.median, np.diff, np.convolve: cálculos matemáticos.
#Es el puente entre OpenCV (que devuelve arrays) y TensorFlow (que espera tensores).}
#tf: Construye modelo (keras.layers, keras.models). Definir datasets (tf.data.Dataset.from_generator). Funciones de entrenamiento (model.fit, class_weight, callbacks).
#es el motor que ejecuta el entrenamiento de la red neuronal.

# ---------- CONFIG ----------
#Uso una variable RAIZ para evita repetir rutas largas asi se facilita mover el proyecto a otra ubicación si se cambiara la raíz.
RAIZ = "/content/drive/My Drive/videoscombi"
RUTA_VALIDOS   = os.path.join(RAIZ, "videosvalidos") #aca el os es para no concatenar strings manualmente (como dato: Windows usa \, Linux/Mac usan /)
RUTA_NOVALIDOS = os.path.join(RAIZ, "videosnovalidos")
CARPETA_RESULT = os.path.join(RAIZ, "resultados") #ruta para guardar la salida del modelo (overlays y csv con resultados de distancia, calificaciones)
os.makedirs(CARPETA_RESULT, exist_ok=True) #crea la carpeta resultados si no existe, porque luego el codigo va a g uardar overlays y CSV en esat carpeta.

#ruta donde se guarda el modelo entrenado
MODELO_SALIDA = "/content/drive/My Drive/modelo_saltos_novalidos_seq.h5"
RUTA_CSV      = "/content/drive/My Drive/reporte_saltos.csv"
#contendrá en juez un dataframe: grupo:nombre de la carpeta/video procesado. p_inv:probabilidad de invalidez. invalido:si fue clasificado como inválido o no. distancia_m:distancia en m. calificacion: etiqueta tipo “normalito”, “muy bien”, “perfecto”.

IMG_SIZE = (160, 160)        # buen equilibrio entre calidad y costo computacional
FRAMES_POR_VIDEO = 24        # 24 frames representativos por salto
BATCH_SIZE = 4               #si tienes vram buena subelo,para que este más rapido el entrenamiento
VAL_SPLIT  = 0.20            # porcentaje destinado a la validación
EPOCHS     = 6               # epocas
SEED       = 42              # determinista (puedes poner la que quieras puse ese número por referencia a La guía del autoestopista galáctico)
ESCALA_METROS = 13.20        # es el factor de conversión a metros porque MediaPipe devuelve posiciones normalizadas en la imagen (x,y entre 0 y 1)
#para la distancia real multiplico por ese factor de escala se fija para que sea consistente con todo el pipeline (dataset, inferencia, juez).
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED) #para que sea 100% reproducible pongo las 3.
#El split estratificado siempre separa los mismos videos en train/val. Los frames seleccionados por muestrear_k serán siempre iguales.
#El modelo empieza con los mismos pesos iniciales en cada ejecución. Dropout aplica las mismas máscaras en entrenamientos repetidos.
#(lo de las mascaras asi se dice cuando se habla de que neuronas se van a apagar)
# ---------- ORDEN / IO ----------
def natkey(s): #orden natural

    return [int(t) if t.isdigit() else t.lower() for t in re.split(r"(\d+)", str(s))] #divide las cedanas en bloques de texto y números.
    #(el (\d+) es para 1 o más dígitos seguidos como esta en paréntesis se devuelven como elementos de la lista)
#filtras archivos que son imágenes con p en minúsculas
def es_imagen(p):
    return str(p).lower().endswith((".jpg",".jpeg",".png")) #revisa la extensión, true si es imagen, false si es otra cosa

def listar_subcarpetas(carpeta): #todas las subcarpetas a una carpeta
    try:
        subs = [os.path.join(carpeta,d) for d in os.listdir(carpeta) # devuelve todos los nombrees de archivos de "carpeta"
                if os.path.isdir(os.path.join(carpeta,d))] # se queda solo con los que son directorios. y construye ruta completa.
    except FileNotFoundError: # si carpeta no existe
        return [] #y devuelve una lista vacía.
    return sorted(subs, key=natkey) #devuelve la lista ordenada de manera natural.

def listar_frames_en_carpeta(carpeta):
    #Filtra overlays/resultados y ordena natural.
    if not os.path.isdir(carpeta): return [] #revisa si existe carpeta y es repertorio, si no regresa una lista vacía.
    fs, ban = [], ("resultado", "_result", "overlay") #en fs es la lista donde se van a guardar ímagenes válidas y ban para imágenes de procesos anteriores.
    for f in os.listdir(carpeta): #f es el nombre del archivo donde se va a iterar sobre todos los nombres de las carpetas
        low = f.lower() #lo pasa a minúsculas para comprobar.
        if not es_imagen(low): continue #usa la función es_imagen(p) para las extensiones .jpg, png, .jpeg y si no es imagen la salta.
        if any(b in low for b in ban): continue #verifica si el nombre del archivo tiene esas palabras, si sí salta.
        p = os.path.join(carpeta, f) #construye la ruta del archivo
        if os.path.isfile(p): fs.append(p) #verifica que sea archivo y no subdirectorio. si lo es, lo agrega a la lista fs.
    return sorted(fs, key=natkey) #devuelve la lista de rutas de ímagenes filtradas y ordena los nombres con el natkey, para orden natural.

def listar_videos(carpeta):
    if not os.path.isdir(carpeta): return [] #ve si carpeta existe y es un directorio y si no, da una lista vacía.
    vids = [] #Inicia una lista vacía donde se guardarán las rutas de videos válidos.
    for f in os.listdir(carpeta): #recorre todos los archivos dentro de la carpeta.
        low = f.lower() #convierte el nombre en minúsculas.
        if not low.endswith((".mp4",".avi",".mov")): continue #filtra por extensión de esos archivo lo demás lo salta.
        if low.endswith("_result.mp4") or low.endswith("_resultado.mp4"): continue #excluye videos con esas terminaciones para no analizar imagenes con texto encima, overlays y resultados.
        p = os.path.join(carpeta, f) #construye la ruta completa del archivo.
        if os.path.isfile(p): vids.append(p) #revisa que sea un archivo y no un directorio, si lo es lo añade a vids.
    return sorted(vids, key=natkey) #Devuelve la lista de rutas de videos válidos ordenados naturalmente.

# ---------- MUESTREO ----------
#fs lista de rutas de frames y k los frames que se usan.
def muestrear_k(fs, k): #Equiespaciado determinista, siempre serán los mismos resultados en la lista fs
    if not fs: return [] #si fs esta vacío regresa una lista vacía
    n = len(fs) #calcula cuantos frames hay de verdad en la carpeta con n el número total de ímagenes en fs.
    #np.linspace(0, max(0, n-1) genera k números equi-espaciados entre ellos, 0 primer índice de la lista el otro es el último índice válido.
    #max asegura que si n=0 (o sea si la lista esta vacía) no de error (aunque ya tenemos el return de arriba), los float pasan a int.
    idx = np.linspace(0, max(0, n-1), num=k, dtype=int) #ya quedaria el array de k índices equiespaciados.
    return [fs[i] for i in idx] #Construye una nueva lista con los frames que salieron de idx, Siempre devuelve exactamente k.

# ---------- PREPROCESO (CONSISTENTE CON TRAIN/INFERENCIA) ----------

def leer_redimensionar(fp):
    im = cv2.imread(fp) #carga la imagen en formato BGR (el estándar de OpenCV).
    if im is None: return None #si por algo no la puede leer, regresa none.
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) #de BGR a RGB porque TF y MobileNetV2 esperan RGB
    im = cv2.resize(im, IMG_SIZE, interpolation=cv2.INTER_AREA) #redimensiona (160,160) e inter para reducción de tamaño, porque promedia píxeles y preserva más información.
    im = im.astype(np.float32) pasa de un uint8(sin signo, entero, 8bits de 0n a 255b y pues es en grises) a un float 32, eso espera TF.
    return preprocess_input(im)  # pasa de [0,255] a [-1,1] #es una función oficial de Keras para MobileNetV2.

# ---------- AGRUPACIÓN ----------
#organizar losdatos en un formato uniforme, muy útil para entrenar.
def armar_grupos(ruta, etiqueta): #donde estan los frames (puede ser subcarpetas o incluso sueltos) y 0 si válido 1 si inválido.
#Devuelve lista de tuplas: (lista_frames, etiqueta, ruta_grupo).
    grupos = []
    for sub in listar_subcarpetas(ruta): #busca las subcarpetas dentro de ruta.
        fs = listar_frames_en_carpeta(sub) #obtiene la lista de frames.
        if fs: grupos.append((fs, etiqueta, sub)) #si la subcarpeta tiene frames (if fs) agrega un grupo con (fs(listaframes encontrados), etiqueta, ruta)
    fs_root = listar_frames_en_carpeta(ruta)  #revisa si en la raíz (ruta) hay frames sueltos, si los hay los mete como grupo
    if fs_root: grupos.append((fs_root, etiqueta, ruta)) #el grupo es de esa forma.
    return grupos #devuelve todos los grupos encontrados.


In [ ]:
# =========================================================
# Celda B — ENTRENAMIENTO
# =========================================================
#tf = base. layers = bloques para construir. models = ensamblador final.
#para sefinir el modelo construir_modelo con models.Model(...).
#Insertar capas con layers.TimeDistributed, layers.GlobalAveragePooling2D, layers.Dense, para eso ocupo estos imports.
import tensorflow as tf
from tensorflow.keras import layers, models

# --- Recolectar grupos ---
#aqui se arma el dataset inicial a partir de los frames que ya tengo en las carpetas.
gr_validos   = armar_grupos(RUTA_VALIDOS,   0) #Busca subcarpetas o frames sueltos en RUTA_VALIDOS y RUTA_NOVALIDOS.
gr_novalidos = armar_grupos(RUTA_NOVALIDOS, 1) ##Devuelve una lista de tuplas (lista_de_frames, y (0 en válido o 1), ruta_del_grupo)
print(f"[INFO] grupos válidos: {len(gr_validos)} | no válidos: {len(gr_novalidos)}") # muestra cuantos grupos de salto hay de cada clase (número de videos representados en tu dataset).
if (len(gr_validos)+len(gr_novalidos)) == 0: #len para saber cuantos elementos tiene la lista, si esa suma es 0 no hay nada.
    raise RuntimeError("No hay frames en las carpetas. Verifica rutas y extensiones.") #para eso sirve raise detiene la ejecución si suma 0.

# --- Split estratificado ---
#separa los train y los de validación cuidando que válidos y no válidos esten representados en cada conjunto (para no tener todos los válidos en train y todos los inválidos en val, por si acaso)
def split_estratificado(V, NV, val_split=VAL_SPLIT, seed=SEED): #(válidos, no válidos, % de validación, para que el barajeo sea reproducible.)
    random.seed(seed)
    V = V.copy(); NV = NV.copy()
    random.shuffle(V); random.shuffle(NV) #Mezclas los elementos de cada lista independientemente, para que no queden igual.
    nV_val  = max(1, int(len(V)*val_split)) if len(V)>0 else 0 #cuantos NV y V van a validación. int es un entero y redondea abajo.
    nNV_val = max(1, int(len(NV)*val_split)) if len(NV)>0 else 0 #el 0 se asegura de que al menos 1 grupo vaya a validación.
    items_val = V[:nV_val] + NV[:nNV_val] #se toman los primeros V y NV para Validación.
    items_tr  = V[nV_val:] + NV[nNV_val:] #lo que sobra se queda en entrebamiento (train)
    random.shuffle(items_val); random.shuffle(items_tr) #mezcla los 2 conjuntos para que no queden primero válidos y luego no válidos.
    return items_tr, items_val #devuelve 2 listas de items con forma (frames, y, ruta) items_tr: dataset de entrenamiento y items_val:dataset de validación.


#aca ya se usa el split, le doy todos los grupos válidos recolectados antes con armar_grupos y los no válidos.
#me da items_tr que son los grupos seleccionados para entrenamiento y items_val los de validación.
# for _,y,_ in items_tr porque solo me interesa Y, o sea si es válido 0 o no 1, if y==0 solo cuenta los grupos válidos. y y==1 no válidos.
#el 1 for... if... Si y==0 devuelve 1 si no, no devuelve nada. (produce una secuencia de 1s tantas veces como haya grupos válidos.)
#el sum, suma todos esos números, cuantos cumplen esa condición y el print tiene  forma de Train V/NV: 2 1 (en el set de entrenamiento hay 2 válidos y 1 no válido.)
items_tr, items_val = split_estratificado(gr_validos, gr_novalidos)
print("Train V/NV:", sum(1 for _,y,_ in items_tr if y==0), sum(1 for _,y,_ in items_tr if y==1))
print("Val   V/NV:", sum(1 for _,y,_ in items_val if y==0), sum(1 for _,y,_ in items_val if y==1))

# --- Generador + tf.data ---
def gen(items_lista, k=FRAMES_POR_VIDEO): #define un generador que reproducira pares (x,y) para alimentar a tf.data.Dataset. (fs,y,ruta, k=24)
    for fs, y, _ in items_lista: #itera cada grupo en una lista de (fs, y, sin la ruta por eso el _ ) lista de frames del grupo y etiqueta (0 válido 1 no válido)
        sel = muestrear_k(fs, k) #agarra k frames de fs (equiespaciado y determinista) si hay menos, muestrear repite bordes hasta k. (para longitud fija)
        batch = [] # para los frames de este grupo
        for fp in sel: #para cada ruta fp seleccionada.
            arr = leer_redimensionar(fp) #Lee la imagen. BGR→RGB. Resize a IMG_SIZE (p.ej. 160×160). float32. preprocess_input de MobileNetV2 → rango [-1, 1].
            if arr is not None: batch.append(arr) #si algo falla, none, lo ignora ese frame.
        if not batch:  # si ningun frame de ese grupo se pudo leer, lo salta.
            continue
        while len(batch) < k:  #si despues de filtrar hay menos de k frames rellena hasta llegar a k.
            batch.append(batch[-1])
        x = np.stack(batch, axis=0)  #mete a k arrays (H,W,3) en un solo tensor (K, H, W, 3)
        yield x.astype(np.float32), np.int32(y) # aca un par (x,y) X en float32 y forma (K,H,W,3) y con Y en int32 y es escalar (0 o 1).

#aquí no aparece el batch porque eso lo agrega tf.data.Dataset.batch(). O sea, antes(24,160,160,3) Después(BATCH_SIZE,24,160,160,3)
#spec_x es para describir la entrada de mi modelo (salto en k frames) la shape es de (24, 160x160, RGB) en float32. x= (24,160,160,3)
#spec_y es la etiqueta,, es escalar() y tiene forma int32 un entero (0 = válido, 1 = no válido).
spec_x = tf.TensorSpec(shape=(FRAMES_POR_VIDEO, IMG_SIZE[0], IMG_SIZE[1], 3), dtype=tf.float32)
spec_y = tf.TensorSpec(shape=(), dtype=tf.int32)

#se crea un Dataset a partir de mi función gen(items_tr). gen hace pares (x,y) con x un tensor (FRAMES_POR_VIDEO, H, W, 3) que son los frames procesados de un salto.
# Y un escalar 0 o 1 indicando válido/no válido, el out es lo que le dice a TF que va a salir.
#hace shuffle en un buffer de tamaño 256 antes de entregarlos. (para que la red no vea primero V y luego NV) seed para que el shuffle sea reproducible.
#el batch(BATCH_SIZE) agrupa ejemplos ya no es un (24,160,160,3) ya seria un (BATCH_SIZE,24,160,160,3) tambien en y (BATCH_SIZE,)
#el .prefetch(tf.data.AUTOTUNE) Mientras el modelo está entrenando en el lote n, TensorFlow ya está preparando el lote n+1 en paralelo.
#eso mejora mucho la velocidad de entrenamiento porque el autotune hace que TF vea número óptimo de lotes para adelantar.
#para ds_val es lo mismo pero sin el shuffle para que sea determinista y estable.
ds_tr  = (tf.data.Dataset.from_generator(lambda: gen(items_tr),  output_signature=(spec_x, spec_y))
          .shuffle(256, seed=SEED).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))
ds_val = (tf.data.Dataset.from_generator(lambda: gen(items_val), output_signature=(spec_x, spec_y))
          .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))

#cuenta cuántos grupos de entrenamiento (items_tr) hay, cada elemento es una tupla (fs,y,ruta) son los que se usan para entrenar, val es lo mismo pero para validación.
print(f"[INFO] train videos: {len(items_tr)} | val videos: {len(items_val)}")

# --- Modelo ---
#defino construir_modelo que regresa una red lista para entrenar.
def construir_modelo(img_size=IMG_SIZE, k=FRAMES_POR_VIDEO, base_trainable=False, lr=1e-3): #(160x160, 24, false para congelar (más rapido y avita SA con pocos datos), la LR)
    inp = layers.Input(shape=(k, img_size[0], img_size[1], 3)) #es la entrada del modelo (24, 160, 160, 3) le voy a la red un video representado como k imágenes RGB.
    #Carga MobileNetV2 sin la “cabeza” (sinx maxpooling ni dropout ni sigmoid), uso pesos preentrenados de imagenet, usamos puro cuerpo aca.
    base = tf.keras.applications.MobileNetV2(include_top=False, input_shape=img_size+(3,), weights="imagenet")
    base.trainable = base_trainable # controla si se copngela la base o se entrena(false rapido, solo capas superiores y true ya fine tuning)
    x = layers.TimeDistributed(base)(inp) #MobileNetV2 a cada frame por separado y sale un mapa de características 2D(5,5,1280) y con los k frames (k, 5, 5, 1280).
    x = layers.TimeDistributed(layers.GlobalAveragePooling2D())(x) #aplica GAP frame-frame convierte (5,5,1280) en un vector (1280,). Ahora x = (k, 1280)  que es un vector de 1280 características por frame.
    x = layers.GlobalAveragePooling1D()(x)#promedia en el tiempo, junta la info de todos los frames (k ejes de tiempo) y Calcula el promedio de los 24 vectores(1280,).
    x = layers.Dropout(0.3)(x)#para evitar overfitting se apaga aleatoriamente neuronas en cada batch.
    out = layers.Dense(1, activation="sigmoid")(x) #lo mismo, calcula la probabilidad de que no sea válido.
    model = models.Model(inp, out) #Define el modelo con su input (inp) y output (out).
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), #adam como optimizador con su lr
                  loss="binary_crossentropy", #esta es la función de perdida y la ocupo porque nuestro problema es binario.
                  metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]) #accuracy % de predicciones correctas. AUC área bajo curva ROC mide qué tan bien separa válidos de no válidos.
    return model #Devuelve el modelo listo para entrenar o evaluar.

model = construir_modelo(base_trainable=False, lr=1e-3) #congelo para entrenar solo capas finales, y el lr es para adam. para tener un tf.keras.Model
model.summary() #resumen tabular.

# --- (Opcional) sanity check: overfit mini ---
#prueba que el pipeline funciona, toma los 2 primeros de válidos y de no válidos.
mini = gr_validos[:2] + gr_novalidos[:2]
if mini: #lista de 4 tuplas. y solo se ejecuta si no esta vacío.
    def gen_mini(): #generador de pares (x,y)
        for fs,y,_ in mini: #itera cada grupo (fs,y,_) de mini
            sel = muestrear_k(fs, FRAMES_POR_VIDEO) #agarra k frames equespaciados y lo garantiza.
            batch = []
            for fp in sel:
#Lee imagen (OpenCV), BGR a RGB, resize a IMG_SIZE(160×160), float32, preprocess_input de MobileNetV2 ([-1,1]).
#Si alguna lectura falla (None), se omite ese frame.
                arr = leer_redimensionar(fp)
                if arr is not None: batch.append(arr)
            if not batch: continue #si ningún frame quedó válido omite ese grupo.
            while len(batch) < FRAMES_POR_VIDEO: batch.append(batch[-1]) #repite el último frame hasta acompletar k.
            x = np.stack(batch, axis=0) #pasa de(H,W,3) a un tensor (K, H, W, 3) este en nuestro caso((24,160,160,3)).
            yield x.astype(np.float32), np.int32(y) #saca pares (x,y) con x:(K,H,W,3) float32 (preprocesado) Y: escalar int32 (0/1).
#Crea un Dataset de TF desde el generador, el out define shapes y dtypes en spec_x: (K,H,W,3) float32. spec_y: () int32 y el batch agrupa de 2 en 2
    ds_mini = tf.data.Dataset.from_generator(gen_mini, output_signature=(spec_x, spec_y)).batch(2) #x:(2, K, H, W, 3) ((2,24,160,160,3)). Y:(2,).
    print("\n[TEST] Overfit rápido a 4 videos…") #mensaje
    #modelop para el test aca descongelo para que el modelo puede memorizar fácilmente 2/4 ejemplos (sanity check).
    model_mini = construir_modelo(base_trainable=True, lr=1e-3) #el LR esta legal para estos procedimientos.
    hist_mini = model_mini.fit(ds_mini, epochs=4, verbose=0) #entrena 4 epocas en donde todo debe subir (loss ↓ accuracy ↑ auc ↑)
    print("acc mini:", float(hist_mini.history["accuracy"][-1]), "auc mini:", float(hist_mini.history["auc"][-1]))#de este test
    #si salio bien, el modelo tiene capacidad para memorizar 4 ejemplos (deberia andar por .9 o más)

# --- class_weight (dar un poco más de peso a 'no válido') ---
#los no válidos que son los menos valen más en la función de pérdida, forzando al modelo a prestarles atención.
#Clase 0 (válidos): peso fijo 1.0. Clase 1 (no válidos): se le da un peso extra:
n0 = sum(1 for _,y,_ in items_tr if y==0) #items es tu lista de grupos del split de entrenamiento (fs,y,ruta)
n1 = sum(1 for _,y,_ in items_tr if y==1) #cuántos grupos hay de cada clase 0 válido y 1 no válido.
cw = None #empiezo cw en none porque por si hay alguna clase vacía, para no aplicar pesos
#cw es un diccionario con pesos para pasar a model.fit(class_weight=cw)
if n0 > 0 and n1 > 0: #solo si hay 1 V y 1 NV se calcula el ratio.
    ratio = n0 / max(1, n1) # cuántos válidos hay por cada no válido.
    cw = {0: 1.0, 1: min(2.0, 1.2*ratio)}  # 1.2–2.0 según desbalance
#1.2 * ratio (para compensar menos no válidos). Se limita con min(2.0, …)nunca se pasa de 2.0.

# --- callbacks ---
#ModelCheckpoint: guarda el modelo durante el entrenamiento.
#monitor="val_auc": se fija en la métrica AUC de validación.
#save_best_only=True: solo guarda el modelo cuando mejora el valor monitorizado (no en cada epoch).
#mode="max": como AUC es mejor cuanto más alto, guarda el modelo cuando aumenta la métrica.
cb = [
  tf.keras.callbacks.ModelCheckpoint(MODELO_SALIDA, monitor="val_auc", save_best_only=True, mode="max"),
  tf.keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=3, restore_best_weights=True)
]
#para early corta el entrenamiento antes si ya no mejora.
#monitor="val_auc": observa la misma métrica AUC de validación.
#mode="max": espera a que mejore
#patience=3: si pasan 3 epochs seguidas sin mejora, se detiene.
#restore_best_weights=True: al parar, restaura los pesos de la mejor epoch (no los últimos). para ahorrar tiempo y sobreajuste

# --- Entrenar ---
#model fit llama a keras aca el modelo recibe datos, ajusta pesos, calcula errores, mejora y guarda.
#ds_tr es el dataset de entrenamiento (tf.data.Dataset)(generadores). Contiene las secuencias de frames con válidos (0) o no válidos (1).
#validation...Es el dataset de validación, para medir qué tan bien se generaliza. para detectar sobreajuste+
#si la precisión de entrenamiento sube pero la validación se queda igual o baja, significa que el modelo memoriza pero no generaliza.
#epochs es el número máximo de pasadas completas por todo el dataset con el cb ES,acaba si no mejora (class_weight=cw)
#le da peso extra a los NV y los cb MC:guarda el mejor modelo según val_auc.ES: corta el entrenamiento si no hay mejoras.
#history guarda los valores de las métricas en cada epoch. (loss, accuracy,auc etc)
history = model.fit(ds_tr, validation_data=ds_val, epochs=EPOCHS, class_weight=cw, callbacks=cb)

#se ejecuta el pipeline: datasets, pesos de clases, validación y callbacks. al final history es para ver cómo evolucionó el modelo.

# guarda la arquitectura del modelo + pesos + configuración de entrenamiento un .h5
model.save(MODELO_SALIDA)
print(f"[OK] Modelo guardado en: {MODELO_SALIDA}")

# --- (Opcional) Fine-tuning corto ---
# base = model.layers[1].layer  # TimeDistributed(base)
# for l in base.layers[:-6]: l.trainable = False
# for l in base.layers[-6:]:  l.trainable = True
# model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
#               loss="binary_crossentropy",
#               metrics=["accuracy", tf.keras.metrics.AUC(name="auc")])
# model.fit(ds_tr, validation_data=ds_val, epochs=3, class_weight=cw, callbacks=cb)
# model.save(MODELO_SALIDA)


[INFO] grupos válidos: 83 | no válidos: 19
Train V/NV: 67 16
Val   V/NV: 16 3
[INFO] train videos: 83 | val videos: 19


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 24, 160, 160,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 24, 5, 5, 1280) │     2,257,984 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_9              │ (None, 24, 1280)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)


[TEST] Overfit rápido a 4 videos…


In [ ]:
# =========================================================
# Celda C — INFERENCIA + JUEZ
# =========================================================
#os para que el “juez” recorra grupos, para localizar videos/frames, y escribir salidas (CSV/overlays).
#cv 2 (opencv) para lectura y escritura de imágenes y videos (cv2.imread, VideoCapture, VideoWriter).
#convierte BGR a RGB, resize, dibuja textos y rectángulos para overlays. (cargar frames,poner cartel Inválido/distancia,exportar video_resultado.mp4.)
#sys detectar entorno o pasarle a subprocess el intérprete actual (sys.executable).
#subprocess en mi pipeline lo uso para instalar dependencias on-the-fly(se descarga instala y corre sobre la marcha, mediapipe) por si no estan.
#time para medir tiempos (time.time()), pausas (time.sleep()), marcar logs, cuánto tarda procesar cada grupo/video.
#estos son un estandar de sistema y utilidades los siguientes son númericos, de ciencia de datos y deep learning.
import os, cv2, sys, subprocess, time
#numpy para np.stack, np.median, np.linspace, np.diff, np.convolve lo ocupo para empaquetar secuencias, suavizar señales (x,y) de MediaPipe y cálculos de métricas/umbrales.
#pandas para Tablas (DataFrames) reporte final juez, junto resultados: probabilidad p_inv, flag invalido, distancia_m, calificacion y guardas a CSV.
#tf carga el modelo entrenado (tf.keras.models.load_model), hacer predict, define CB. También para tipos (tf.float32) y utilidades.
import numpy as np, pandas as pd, tensorflow as tf
#para cachear resultados y hacer memoización (si usas mucho una funcion no se calcula, se guarda y se ocupa)
#Lectura + preprocesado varias veces. Listados de carpetas si consultas repetidamente,evita recomputación,acelera juez cuando hay reutilización.
from functools import lru_cache
#from tensorflow.keras.applications.mobilenet_v2 import preprocess_input Normalización oficial de MobileNetV2:
#convierte píxeles a float32 en rango [-1, 1].para que la inferencia sea coherente con el entrenamiento (misma escala de entrada).
#se aplico en leer_redimensionar(fp) antes de pasar al modelo.
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# ---------- PARÁMETROS ----------
UMBRAL_INVALIDO  = 0.57     # baja un poco para subir recall (más no válido, si da muchos NV, subir)
MARGEN_OK        = 0.04     # banda dudosa estrecha (antes 0.10 mientras más alto más dudoso)
N_BATCH          = 16       # grupos por batch en GPU/TPU
GENERAR_OVERLAYS = False    #activar al cierre o para compartir resultados visuales; mantenlo en False en desarrollo para iterar rápido. (T para render)
K                = FRAMES_POR_VIDEO #los 24 equiespaciados que representan el salto
SIZE             = IMG_SIZE #160x160

# ---------- CARGA MODELO ----------
#carga un modelo previamente entrenado desde disco. (.h5 acá)
clf_seq = tf.keras.models.load_model(MODELO_SALIDA) #ruta donde se guardo en la fse de entrenamiento(MODELO_SALIDA = "/content/drive/My Drive/modelo_saltos_novalidos_seq.h5")
#clf_seq = classifier sequence (clasificador de secuencias) (se usará en p = clf_seq.predict(x))
#La arquitectura del modelo (capas y conexiones).
#Los pesos entrenados (valores numéricos aprendidos).
#La configuración de compilación (optimizador, pérdida, métricas) para generalizar.

# ---------- PREPROCESAMIENTO (igual que en entrenamiento) ----------
#si se usa el mismo frame varias veces (o en distintos muestreos), ya no lo vuelve a leer ni procesar(acelera mucho la inferencia.)
@lru_cache(maxsize=300)#los resultados de esta función se guardan en memoria caché.(se recordarán hasta 300 llamadas distintas en fp)
#aca empiezo con _ el nombre para recordarme a mi mismo que (o a alguien más) es de uso interno.
#(no deberia usarla directamente desde fuera porque podría cambiar o eliminarse sin previo aviso es un detalle interno es como si fuera privada)
def _load_preproc(fp): #recibe fp (file path) o seala ruta del archivo de imagen que se quiere procesar.
    im = cv2.imread(fp) #lee desde el disco en formato bgr.
    if im is None: return None #si falla regresa none.
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) #paso de bgr que usan open cv a rgb que usa TF
    im = cv2.resize(im, SIZE, interpolation=cv2.INTER_AREA) # 160x160 como en entrenamiento I_A reducciones de tamaño es más preciso y suave.
    im = im.astype(np.float32) #Convierte los valores de píxeles de uint8 (0–255) a float32 por keras y MobileNet necesitan float.
    return preprocess_input(im)  # normaliza los píxeles[-1,1] para que inferencia se igual en distribución a lo que se uso en entrenamiento.
#resumen: Lee la imagen la pasa a RGB.
#La escala a (160,160).
#La convierte a float32.
#La normaliza a [-1,1].
#Devuelve un tensor listo para pasar al modelo y por lru_cache si lo ocupo de nuevo esta a la mano, (Devuelve un numpy array con forma (H,W,3).)


#otro helper interno en el pipeline de inferencia, sel es lista de rutas de imágenes ya seleccionadas (24 frames equiespaciados de un salto).
def _stack_from_sel(sel):
    batch = [] #vacía pero se irá llena con los frames preprocesados
    for fp in sel: #recorre cada filepath en sel
        arr = _load_preproc(fp)
        if arr is not None: batch.append(arr) # Si arr no es None (imagen válida), la mete en el batch.
    if not batch: return None # si por alguna razon ninguna imagen es válida da un none, para no romper nada más adelante.
    while len(batch) < K: batch.append(batch[-1]) #se asegura que hay k frames, rellenando con el último válido.
    return np.stack(batch, axis=0).astype(np.float32)#np...0 mete la lista de arays a un solo tensor (24, 160, 160, 3), se mete al modelo este tensor.

def _sel_k(fs): #la entrada es fs (lista de rutas de frames)
    n = len(fs) #n calcula cuantos frames hay en la lista si la carpeta tiene 200 ímagenes pss n=200
    idx = np.linspace(0, max(0, n-1), num=K, dtype=int) #k Nu.Eq entre 0 y n-1. redondea a enteros y cuida si n=0 seria 0 el tope y no -1.
    return [fs[i] for i in idx] #construye una nueva lista con las rutas a esos frames seleccionados.

# ---------- CLASIFICACIÓN EN BATCH ----------
#una función para clasificar varios grupos de frames (un grupo = un salto). de entrada tiene a grupos.
#lista de tuplas tipo (nombre, carpeta) que vienen en el recolector (armar_grupos).
def clasificar_grupos(grupos):
    preds = {} #para guardar las predicciones finales, con clave = carpeta y valor = probabilidad p_inv.
    X, meta = [], [] #X es la lista acumuladora de tensores de cada grupo, meta: guardar metadatos (nombre, carpeta) en el mismo orden que X.
    for (nom, car) in grupos: #nom=nombre del grupo ("nv1") car=ruta de la carpeta donde están los frames de ese grupo.
        fs = listar_frames_en_carpeta(car) #fs = lista de frames en la carpeta.
        if not fs: #si esta vacia no se puede procesar.
            preds[car] = None #se guarda None en preds[car] y pasas al siguiente grupo.
            continue
        sel = _sel_k(fs) #selecciona k frames equiespaciados.
        x = _stack_from_sel(sel)#aplica esos frames a un tensor (K,H,W,3). (24,160,160,3)
        if x is None:
            preds[car] = None #si algo falla, regresa none en predicciones y salta.
            continue
        X.append(x) #se guarda el tensor del grupo (x) en la lista X, guardo los metadatos (nombre, carpeta) en meta.
        meta.append((nom, car)) #Ahora ambos están sincronizados: X[i] corresponde a meta[i].
        if len(X) == N_BATCH: #cuando se acumulan el número de batchs necesarios se procesa.
            xb = np.stack(X, axis=0)  # junta la lista en un tensor 5D de forma(B,K,H,W,3) b es el tamaño del batch
            pb = clf_seq(xb, training=False).numpy().reshape(-1) #ejecuta el modelo en modo inferencia(da probas)convierte a numpy y aplana a vector 1D.
            for (n,c), p in zip(meta, pb): preds[c] = float(p) #Asocia cada carpeta c con su probabilidad p.
            X, meta = [], [] #Limpias X y meta para acumular el siguiente lote.

#Tras salir del bucle, puede que queden grupos pendientes menores a N_BATCH.
#Este bloque asegura que esos también se procesen.
#X es una lista de tensores (K,H,W,3), uno por grupo. np.stack...0 los junta por eso ya ES 5D.
#clf_seq espera entrada (B,K,H,W,3). Procesa cada grupo de frames y produce una probabilidad por grupo (salida de la capa Dense(1, sigmoid)) (B,1) como salida.
# de (B,1) reshape(-1) aplana a (B,) (de matriz a vector) para recorrerlo rápido meta guarda la info (nombre, carpeta).
#pb guarda las probabilidades (ya en un array plano). Con zip(meta, pb) para  ambos simultaneo y asignar las predicciones a cada archivo.

    if X: #si el número de grupos no es multiplo del batch (16) pon tu 34=16+16+2 pues eso 2 los captura, los que sobrarian de X.
        xb = np.stack(X, axis=0) #junta lo que queda en formato (len(X), K, H, W, 3)
        pb = clf_seq(xb, training=False).numpy().reshape(-1) #calcula las probs y el .reshape da un vector de probabilidad por grupo.
        for (n,c), p in zip(meta, pb): preds[c] = float(p)
    return preds #Devuelves un diccionario con las predicciones.

#en el for pb es de probs de pb = clf_seq(...).numpy().reshape(-1), zip los une para recorrerlos en paralelo, for desempaqueta(n,c) y p de prob.
#zip para empareja cada carpeta con su probabilidad y se mete al dic de "preds" la clave es carpeta de frames (ruta del salto).
#valor es la probabilidad de ser "no válido" (recordar que la red esta entrenada con sigmoid).

# ---------- MEDIAPIPE (solo para válidos con margen) ----------
#es la parte del postprocesado con mediapipe solo para vídeos válidos y con probabilidad clara, no en el aspectro dudoso (se definio en MARGEN_OK)
try:
    import mediapipe as mp #intenta instalar mediapipe, si esta instalada no pasa nada, es por seguridad.
except ModuleNotFoundError: #si no esta instalada se lanza la excepción
    subprocess.check_call([sys.executable, "-m", "pip", "install", "mediapipe==0.10.14", "--quiet"]) #aca se usa al interprete para instalar pip.
    import mediapipe as mp #una vez instalado importa mediapipe (est bloque es de seguridad lo puedes quitar si tienes mediapipe)

mp_pose       = mp.solutions.pose #es la parte de mediapipe que sirve para el esqueleto humano (pose estimation).(se uso para analizar los saltos válidos:verificar técnica y postura.)
FUENTE        = cv2.FONT_HERSHEY_SIMPLEX#se usa en OpenCV (cv2.putText) para los o0verlays (“VÁLIDO”, coordenadas, ángulos, etc.).
FPS_SALIDA    = 30 #define los frames por segundo de los videos de salida que se generan al exportar con overlays.(fijos a 30 FPS).

#para calcular un punto representativo del pie a partir de los landmarks que da MediaPipe Pose. (notas que las funciones que empiezan con "_" son como una función privada, no le menees a menos que sepas exacatamente que haces, pero asi ya funciona bien.)

def _xy_pie_mediana(landmarks): #es la lista de puntos clave de mediapipe y esta en coordenadas normalizadas (x,y,z)(z es profundidad peor no en m, más respecto al cuerpo, o sea si te salen valores negativos la cámara esta adelante.)
    cand = []
    for lm in (mp_pose.PoseLandmark.LEFT_HEEL, #talon izquierdo.
               mp_pose.PoseLandmark.RIGHT_HEEL, #talon derecho
               mp_pose.PoseLandmark.LEFT_FOOT_INDEX, #punta del pie izquierdo.
               mp_pose.PoseLandmark.RIGHT_FOOT_INDEX): #punta del pie derecho.
        L = landmarks[lm]; cand.append((L.x, L.y)) #saca sus coordenadas (x,y) y las guarda en cand
    xs, ys = zip(*cand); return float(np.median(xs)), float(np.median(ys)) #Se separan las coordenadas X e Y, se calcula media de x e y, tenemos un punto central robusto del pie.
#use la media porque es menos sensible a valores atípicos, para que sea más robusto.

# aca es lo del pie promedio que vimos arriba en un bucle que procesa muchos frames.

def extraer_xy_de_frames(rutas_frames):
    xs, ys = [], [] #aca se guardan todas las coordenadas Y e X del pie medio (lo de arriba) frame a frame.
    ##pose es el modelo de MP que detecta landmarks del cuerpo, static... para secuencias de video, confianza mínima para deterctar la pose y
    with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose: #confianza mínima para seguir landmarks a lo largo de un video.
        for fp in rutas_frames: #Itera sobre la ruta de frames.
            img = cv2.imread(fp) #carga la imagen desde disco (drive)
            if img is None: continue #si no se puede cargar, pasa la siguiente.
            rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #pasa de bgr a rgb.
            res = pose.process(rgb) #regresa un objeto con la pose detectada ya con los landmarks detectados en ese frame.
            if res.pose_landmarks: #revisa si MP pose encontro puntos del cuerpo, si si pues da los 33 landmarks
                x, y = _xy_pie_mediana(res.pose_landmarks.landmark) #llama esa función que es para el pie robusto en (x,y)
                xs.append(x); ys.append(y) #guarda los X y los Y en listas separadas.
    return xs, ys #regresa las trayectorias. (cada (xᵢ, yᵢ) corresponde al pie en un frame.)

#análogo de extraer_xy_de_frames, pero no recibe una lista de rutas (fs) de imágenes ya guardadas, recibe un único video (path_video) y va leyendo frame por frame con OpenCV.
#fs listra completa de frames, lista de rutas a frames.
#fp frame individual (una ruta de la lista ), o sea la ruta de un solo video.
def extraer_xy_de_video(path_video):
    xs, ys = [], [] #lista donde se acumulan las coordenadas X, Y de los pies. (se guarda la serie de posiciones (en tiempo) de los pies.)
    with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose: # el false dice video en movimiento,optimiza el tracking usando frames previos del vídeo.
        cap = cv2.VideoCapture(path_video) #cap es un objeto de opencv permite leer frame a frame y abre el archivo de video.
        while True:
            ok, frame = cap.read() #ok si leyo un frame de video y cap.read() pues lee un video. (cuando no quedan frames se corta el bucle.)
            if not ok: break
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #pasa de BGR a RGB porque asi lo pide mediapipe.
            res = pose.process(rgb) #analiza ese rame con el modelo de pose
            if res.pose_landmarks:#si detecta una pose tomas los landmarks de pies con tu función _xy_pie_mediana. (pie robusto)
                x, y = _xy_pie_mediana(res.pose_landmarks.landmark)
                xs.append(x); ys.append(y) #guarda las coordenadas en las listas xs y ys (por separado).
        cap.release() #ya se termino de usar pues se cierra. (también para libera memoria que usaba Opencv)
    return xs, ys #devuelve las coordenadas que se sacan del video.

#este arreglo es para eliminar ruido, picos bruscos en las coordenadas de los pies a lo largo del tiempo, para que la curva sea más suave.
#al sacar promedio se reduce el ruido. k (ventana) es ponle que es 3, toma 3 valores seguidos, saca promedio y lo usa como valor de suavizado.
#media movil es la tecnica de suavizado más simple(no hace falta más) tomas la ventana k d evalores seguidos, promedio, pasas de ventana y asi.
# el ker es un kernet uniforme de longitud k, si es 3 seria [1/3, 1/3, 1/3] y sirve para sacar el promedio de cada ventana de k elementos.
#valores de suavizado pequeño borran poco ruido (la señal es parecida) si es grande se van los picos pero la señal se hace lenta, tarda en reaccionar a cambios bruscos.
#con k=11, cada punto se promedia con sus 10 vecinos más cercanos (5 a la izquierda y 5 a la derecha)cada punto se reemplaza por el promedio de sus k vecinos.

def _suavizar(v, k=11):   # cambia a como mejor te convenga, pero yo lo hice así.(cuando hablo asi, me refiero al lector si alguien lo lee)
    if len(v) < 3: return np.array(v, float) #si el vector v solo tiene menos de 3 valores no lo suaviza, no tiene sentido, lo regresa tal cual.
    k = max(3, int(k) | 1); ker = np.ones(k)/k #k es el tamaño de la ventana de suavizado, el 1 para que sea impar(media móvil se centra mejor con ventanas impares) y 3l max 3 para que no sea menor que 3.
    return np.convolve(np.asarray(v, float), ker, mode="same") #np.convolve aplica el kernel sobre la señal v y mode devuelve un vector de la misma longitud que la entrada.

#ys y xs son las coordenadas normalizadas(0 a 1) de los pies (pie robusto) el tau es la sensibilidad para ver si es un salto(uso el valor default). o sea que es el umbral base de la velocidad vertical para decidir cambios claros (despegue/aterrizaje).
#luego se suaviza para trabajar con x e y ya suavizados, con media movíl y ventana 11, para eliminar ruido de Mediapipe.
#si en y (vertical)son menos de 11puntos regresa none, si el salto es muy pequeño no detecta, no seria confiable.
#punto más alto del pie en y es el menor porque en C normalizadas y crece para abajo. y vy ≈y[t+1] - y[t]. velocidad vertical.
#y(t) seria como la posición del frame en t y si lo vemos como limite Δt=tiempo entre frames ( 30 fps, Δ𝑡=1/30 Δt=1/30 s).  pero como es contante y abusando muchísimo del lenguaje sería como una derivada discreta.
#np.abs toma el valor absoluto porque solo quiero ver que tan duro se mueve, le saco la mediana, si no hay datos 0.0
#el tau el es estandar y 0.5*med es la mitad de la mediana del movimiento real (se usa el mayor de los dos)
#si el salto es muy suave (valores pequeños), se usa el mínimo tau.
#Si el salto es más fuerte (valores grandes), se sube el umbral a 0.5*med.
#o sea se calcula la mediana de la velocidad. saca la mitad 0.5*med y si la mitad de la mediana es mayor que tau, usamos 0.5 * med.
#Si 0.5*med es menor que tau, usamos tau, para que no baje mucho el umbral y confundir ruido con salto.
#antes toma la velocidad vertical (vy) antes del ápice(punto máximo) apex-1 para mirar solo la fase de subida. si apex es muy pequeño 0 o 1 al inicio del video apex-1 sería 0 o negativo.
#max(apex-1, 1), para que al menos tomemos hasta el índice 1.nunca se pide una porción de tamaño 0 (da problemas al buscar candidatos).
#cand_take busca los frames antes del ápice donde la velocidad vertical es muy negativa vy < -tau_eff el pie sube con claridad o sea que devuelve los candidatos a despegue.
#take=cand_take si hay candidatos, toma el último ([-1]) la última vez que el despegue es fuerte, la asocio al despegue.
#si no los hay usa un fallback heurístico apex-4 (4 frames antes del apendice acotado a ≥ 0) es el plan B si no hay despegue claro (vy < -tau_eff)
#supone que el despegue ocurrió unos 4 frames antes del punto más alto. (el video es a 30 fps, serian de unos 0.13 segundos antes del ápice.)
#en después es la parte de la bajada, cand... ve frames en los que la velocidad vertical es positiva (vy > tau_eff) osea que el pie está bajando con fuerza y devuelve los índices donde ocurre esa condición.
#si hay candidatos en land=apex... ( o sea cand_land.size > 0) usa el primer(cand_land[0] el primer cruce donde la bajada es clara) y si no hay, para eso esta el +4 o sea que el aterrizaje se dio 4 frames después del apex.
#len(x)-1 si x tiene la misma longitud que tu serie temporal (e,j100 frames), índices válidos de 0 hasta 99. Así que len(x)-1 es el último índice válido.
#min(take, len(x)-1) asegura que take no sea mayor que el último frame. (e.j take=101 lo pasa a 99).
#max(0, min(...))asegura que take nunca sea menor que 0 (e.j take=-6 lo pasa a 0).El resultado siempre esta entre 0 y len(x)-1. Lo mismo para land.
#en el if es porque en un salto válido el despegue debe ocurrir antes que el aterrizaje para asegurar más.
#take = el frame del despegue (cuando el pie se separa del suelo).
#land = el frame del aterrizaje (cuando el pie vuelve a tocar el suelo).
#la función devuelve un segmento de salto como una pareja de índices donde empieza y termina el salto en frames.

def _segmento_salto(xs, ys, tau=0.002):
    x = _suavizar(xs, 11); y = _suavizar(ys, 11)
    if len(y) < 11: return None
    apex = int(np.argmin(y)); vy = np.diff(y)
    med = np.median(np.abs(vy)) if len(vy) else 0.0; tau_eff = max(tau, 0.5*med)
    antes = vy[:max(apex-1,1)]; cand_take = np.where(antes < -tau_eff)[0]; take = cand_take[-1] if cand_take.size else max(apex-4, 0)
    despues = vy[apex:];       cand_land = np.where(despues >  tau_eff)[0]; land = apex + (cand_land[0] if cand_land.size else 4)
    take = max(0, min(take, len(x)-1)); land = max(0, min(land, len(x)-1))
    if land <= take: return None
    return take, land

#mide que tan lejos se movio el pie durante el salto, basicamente calcula la distancia.

def calcular_distancia_solo_segmento(xs, ys, escala=ESCALA_METROS): #lista de arrays con las coordenadas normalizadas del pie en cada frame y la escala son metros reales de la anchura normalizada 1.0
    if not xs or not ys: return None #si las listas estan vacias regresa none (no se pudo calcular distancia)
    x = _suavizar(xs, 11); seg = _segmento_salto(xs, ys) #suavizado a xs  (horizontal) Intenta detectar despegue y aterrizaje con _segmento_salto.
    if seg is not None: #si se detecto un segmento (salto y aterrizaje) seg = (take, land)
        i0, i1 = seg; dx = float(x[i1] - x[i0]) #i0 = índice de despegue, i1 = índice de aterrizaje y vecuánto avanzó horizontalmente el pie (dx = x[i1] - x[i0])
        if dx <= 0: dx = float(np.max(x) - np.min(x)) #si por algo se invierte el orden o ruido (como fallback mide todo el rango horizontal (max - min).)
    else: #Si no se detectó el segmento del salto.
        dx = float(np.max(x) - np.min(x)) #usa la distancia horizontal como estimación (peor pero seguro).
    return abs(dx)*float(escala) #da distancia en metros: dx es en coordenadas normalizadas [0–1].
    #Multiplica por escala (13.20 m este valor es ajustable, prueba como funciona mejor, yo por limitaciones ya no puedo seguir probando, pero es aledaño).
    #abs() valor positivo (valor absoluto).

def calificacion(dist): #distancia del salto en metros (float) o none si no puede calificar
    if dist is None: return "No detectado" #si no hay valor por lo que sea, regresa no detectado.
    if dist < 6.0:   return "normalito" #si la distancia es menor a 6m, normalito.
    if dist < 7.5:   return "muy bien" #menor a 7.5 muy bien (son los que más hay, lo que concuerda con la vida real)
    if dist < 8.5:   return "Muy Bueno" #menor a 8.5 muy bueno. (califica 10 así, en la vida real eran 5)
    return "perfecto" #si se pasa de 8.5 regresa perfecto (en la vida real hay 6 arriba del 8.5, califico 17 asi, se pasa por 11, pero es corregible con los valores que te mencione antes, no es que no sirva la red, faltan intentos.)

# ---------- Heurística para decidir en banda dudosa ----------
#El objetivo es decidir si un salto debe considerarse inválido aunque la red neuronal no esté 100% segura.
def heuristica_invalido(xs, ys): #defino una función que recibe xs y ys
    xs = np.asarray(xs); ys = np.asarray(ys) #pasa xs y ys en arrays de np(para usar .max(), .min(), operaciones vectorizadas y así).
    cov = len(xs) / max(1, FRAMES_POR_VIDEO)# mide el % frames con pose (k=24 y xs tiene longitud 10, detección en 10/24 = 0.41 (41%)) el max evita 0/x si k=0.
    dx  = (xs.max()-xs.min()) if xs.size else 0.0 #desplazamiento horizontal del pie (diferencia máximo y mínimo en X).
    dy  = (ys.max()-ys.min()) if ys.size else 0.0 #en vertical, si(ys.size == 0), devuelve 0.0 en vez de intentar max() o min()(error).
    flags = 0 #contador de alertas (si el% de frames detectados es bajo 1 alerta, si casi no se mueve horizontalmente o verticalmente otro y así)
    if cov < 0.40:  flags += 1
    if dx  < 0.020: flags += 1
    if dy  < 0.015: flags += 1
    return flags >= 2 #si se cumplen 2 de esas condiciones se considera inválido (heurísticamente, o sea no sirve.)

# ---------- Overlays ----------
#frames lista de rutas a imágenes (frames de un video).
#texto es el texto que se va a escribir sobre cada frame.
#color es el color del texto en formato BGR ((0,255,0) = verde).
#out_path es la ruta donde se guardará el video final.
#out=cv2... crea un escritor de video (VideoWriter). cv2.Vid...mp4v es para guardar en MP4, luego los fps de salida,(w, h) resolución del video.
#escribe el texto en el cuadro negro Posición (40,65) (empieza el texto).
#Fuente : FONT_HERSHEY_SIMPLEX.
#Tamaño de letra → 1.1.
#Color el que le pongas.
#Grosor → 2.
#LINE_AA para el suavizado de bordes (para que se vea mejor).
#como dato (0,0,0) es negro y el -1 es para que se relllene de ese color.
def overlay_frames(frames, texto, color, out_path):
    if not GENERAR_OVERLAYS or not frames: return #si no hya frames sale sin hacer nada.
    first = cv2.imread(frames[0]); #Carga el primer frame para conocer el ancho (w) y alto (h) de las imágenes.
    if first is None: return
    h, w = first.shape[:2] #para inicializar el video correctamente. (el 2 es porque no agarro coloer, solo h y w alto y ancho)
    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*"mp4v"), FPS_SALIDA, (w, h))
    for fp in frames: #itera sobre todos los frames
        img = cv2.imread(fp) #le cada imagen.
        if img is None: continue #si no puede leer salta.
        x2 = min(w-20, 40 + int(0.6*w)) #es para la caja de texto que saldra en el video, se llama ancho dinámico
        cv2.rectangle(img, (30, 20), (x2, 80), (0,0,0), -1) #dibuja un rectangulo negro, en la parte superior (para que se vea el texto)
        cv2.putText(img, texto, (40, 65), cv2.FONT_HERSHEY_SIMPLEX, 1.1, color, 2, cv2.LINE_AA)
        out.write(img) #agrega el frame modificado (con overlay) al archivo de video.
    out.release() #cierra el archivo de video y guarda el resultado.

#w-20 es el ancho del video menos 20 píxeles y 40 + int(0.6*w) el valor que calculamos antes.
#elige el menor de los dos, para asegurarse de que el rectángulo no se pase del borde derecho.
#el min es para que jamas dibuje afuera del rectangulo y se estira para que se vea mejor.
#nota :Rojo usado cuando el modelo/heurística decide que el salto es inválido. Verde es usado cuando pasa como válido y se mide la distancia.

def overlay_video(in_path, texto, color, out_path): #frame a frame tipo st y escribe otro video con clasificación superpuesta (entrada, (cadenaN/NV), color, Ruta de guardado.
    if not GENERAR_OVERLAYS: return #igual que antes si es False,no hace nada (sirve como switch para no gastar tiempo en renderizado).
    cap = cv2.VideoCapture(in_path) #abre el archivo de video.
    if not cap.isOpened(): return #devuelve False o sea que sale sin hacer nada.
    fourcc = cv2.VideoWriter_fourcc(*"mp4v"); fps = cap.get(cv2.CAP_PROP_FPS) or 30 #codec para mp4, fps del video de entrada(30), si no se detecta igual usa 30.
    w, h = int(cap.get(3)), int(cap.get(4)) #ancho(3) y alto del video(4) (son IDs de propiedades predefinidas en OpenCV)
    out = cv2.VideoWriter(out_path, fourcc, float(fps), (w, h)) #para guardar, ruta y nombre de salida, códec, los fps que tendra, tamaño de fotograma (ancho, alto)
    while True: #buble para leer cada frame el video original.
        ok, frame = cap.read() #si pudo leer fotogramas, ok (como dato los frames son matriz numpy porque uso opencv)
        if not ok: break #se sigue leyendo frames hasta que no hay más.
        x2 = min(w-20, 40 + int(0.6*w)) #ancho (se saco de cap.get(3)) 60% del ancho del video, le sumas un margen de 40 píxeles(el rectángulo se hace un poquito más ancho hacia la derecha)
        cv2.rectangle(frame, (30, 20), (x2, 80), (0,0,0), -1) #dibuja un rectángulo negro arriba. (es lo mismo que le otro rectangulo)
        cv2.putText(frame, texto, (40, 65), cv2.FONT_HERSHEY_SIMPLEX, 1.1, color, 2, cv2.LINE_AA)#escribe el texto (ej:"Inválido (p=0.72)") en el color que le toca.
        out.write(frame) #agrega el frame procesado (ya con texto) al video de salida.
    cap.release(); out.release() #cap.release(): libera el video de entrada y out.release(): guarda y cierra el video de salida.

# ---------- MAIN ----------
#aca se empiezan a alistar los grupos que se van a evaluar, es el main de inferencia.
t0 = time.time() #como referencia para ver cuánto tardó todo el proceso de entrenado (guarda el tiempo actual o sea los segundos desde el epoch)
grupos_val   = [(os.path.basename(p), p) for p in listar_subcarpetas(RUTA_VALIDOS)] #devuelve las subcarpetas de válidos, cada subcarpeta p es una tupla, con nombre de la carpeta y la ruta completa de la subcarpeta que resulta en: grupos_val es una lista de tuplas.
grupos_noval = [(os.path.basename(p), p) for p in listar_subcarpetas(RUTA_NOVALIDOS)] #lo mismo lista de tuplas con (nombre, ruta) para los grupos de no válidos.
todos = grupos_noval + grupos_val #se concatenan las listas para tener todos los videos V y NV. (se recorre para clasificación (clasificar_grupos))
print(f"[INFO] grupos totales: {len(todos)} (V={len(grupos_val)}, NV={len(grupos_noval)})") #grupos totales, válidos y no válidos.

# 1) clasificar todos rápido
#llama a tu función clasificar_grupos (se definio antes). Le pasa la lista "todos", que contiene tuplas con (nombre, ruta) de cada grupo de frames.
#la función hace este flujo interno: Para cada carpeta de frames: Lista las imágenes con listar_frames_en_carpeta.
#selecciona K frames equiespaciados (_sel_k).
#preprocesa cada frame con _stack_from_sel lo que lo convierte en tensor de forma (K, H, W, 3).
#agrupa varios grupos hasta llegar a N_BATCH (16 por defecto) y forma un batch tensor 5D de forma (B, K, H, W, 3).
#pasa ese batch por el modelo clf_seq para obtener las probabilidades de ser inválido (p_inv).
#pasa el resultado de salida (un tensor de probas) a un vector NumPy 1D con .numpy().reshape(-1).
#y ya por último va guardando los resultados en un diccionario.
#pdict es un diccionario cuyas keys son las rutas de los grupos y sus values(datos de una variable) son las probas de ser inválidos.
#aquí no se hace heurística, ni overlays, ni cálculos de distancia. Solo el modelo en modo batch. (mucho más eficiente que ir uno por uno)
#Entrada: lista "todos" con los grupos.
#Proceso: preprocesar, batch, modelo, proba.
#Salida: diccionario pdict con la probabilidad de invalidez por cada carpeta de frames.

pdict = clasificar_grupos(todos)

#Segundo paso de la inferencia: después de obtener las probabilidades (pdict), ahora vemos qué hacer con cada salto
#Si es inválido rojo + texto “Inválido”. Si es válido se usa MediaPipe para calcular distancia y dar calificación.
#Aquí todavía no se decide, solo se prepara la info base (p, p_txt, out_name) para cada grupo.
#p_txt tambien es para para superponer en overlays y para imprimir logs.
#out_name define el nombre del archivo de salida donde se guardará el video con overlay (el marcador de válido/inválido o distancia).
#CARPETA_RESULT es la carpeta central de resultados (/content/drive/My Drive/videoscombi/resultados/elnombredetuvideo_resultado.mp4)


resultados = [] #un diccionario con salida amigable para leer lo que pasa ({"grupo": ruta, "p_inv": p, "invalido": True/False, "distancia_m": 7.2, "calificacion": "Muy Bueno"})
# 2) procesar según decisión; MediaPipe solo para válidos con margen
for nom, car in todos: #itera sobre cada grupo de "todos" nom:nombre de la carpeta y car=ruta completa de esa carpeta.
    p = pdict.get(car, None)#ve en pdict (dic de probas) el valor de esa ruta car, devuelve None si no encuentra.p es la proba que ese grupo sea inválido.
    p_txt = "N/A" if p is None else f"{p:.2f}" #pasa la probabilidad de p en texto bonito, si p=None, escribe "N/A". Si sí hubo,redondea a 2 decimales (0.8231 → 0.82).
    out_name = os.path.join(CARPETA_RESULT, f"{nom}_resultado.mp4")

    # Decisión rápida
    #aca empieza y ala decisión binaria.
#if p is none...:se decide si el salto es inválido: p is None: no hubo predicción, asume inválido (porque no se pudo evaluar bien).
#p >= UMBRAL_INVALIDO: la probabilidad supera el umbral → se clasifica como inválido.
#UMBRAL_INVALIDO es el corte o sea  0.57. Si la red cree con ≥ 57% que es inválido es inválido.
#if GENERAR es por si se pide generar overlays (True), se crea un video con la etiqueta. listar_frames_en_carpeta(car)recoge los frames del grupo.
#overlay_frames(..., f"Invalido (p_inv={p_txt})",...)dibuja en cada frame: Texto: "Invalido (p_inv=0.85)"
#color: (0,0,255)rojo en formato BGR de OpenCV, guarda como archivo en out_name.
#en resultados mete un registro a la lista de "resultados"con: "grupo": car siendo la ruta del grupo. "p_inv": p la proba de inválido.
#"invalido":True que marcado como inválido. "distancia_m": None si no se mide distancia porque no tiene sentido en inválidos.
#"calificacion": "Invalido" → etiqueta textual.
# la decisión: Si un salto es claramente inválido (o ni siquiera se pudo evaluar bien) se marca y no gastas más tiempo.
#Solo los que pasan esta condición siguen en el bucle para cálculo de distancia y calificación.


    if (p is None) or (p >= UMBRAL_INVALIDO):
        if GENERAR_OVERLAYS:
            overlay_frames(listar_frames_en_carpeta(car), f"Invalido (p_inv={p_txt})", (0,0,255), out_name)
        resultados.append({"grupo": car, "p_inv": p, "invalido": True,
                           "distancia_m": None, "calificacion": "Invalido"})
        print(f"{nom}: Invalido (p_inv={p_txt})") #log en consola.
        continue #pasa al siguiente grupo sin ejecutar todo el bucle. o sea que si es inválido el grupo, no pasa a MediaPipe ni a calcular distancias.

    # Banda dudosa correr pose y decidir con heurística
#recordar que UMBRAL_INVALIDO = 0.57 y MARGEN_OK = 0.04. Por eso cualquier predicción entre 0.53 y 0.57 entra en esta condición.
#la idea es que si la probabilidad está cerca del umbral, no decides solo con la red, sino que aplicas un chequeo adicional.
#si existe un video (vd no está vacío) se lo analiza con extraer_xy_de_video.
#Si no hay video pero sí frames se usas extraer_xy_de_frames.
#Resultado: xs = coordenadas horizontales (x) del pie a lo largo de los frames. ys = coordenadas verticales (y) del pie a lo largo de los frames.
#ifheuristica...: Aplicas la heurística (que hicimos): Cobertura muy baja de frames con pose.
#Movimiento horizontal (dx) muy pequeño.
#Movimiento vertical (dy) muy pequeño. Si al menos 2 condiciones se cumplen se marca como inválido.
#es como un  mini juez extra: Si la red duda, se usa MediaPipe para observar el movimiento del pie.
#Si el movimiento es bajo, se fuerza la etiqueta de inválido. Si el movimiento sí parece salto, se le da chance de medirse como válido.

    if p >= UMBRAL_INVALIDO - MARGEN_OK:
        fr = listar_frames_en_carpeta(car); vd = listar_videos(car) #carga los frames del video completo del grupo y prepara los datos para MPipe y sacar las coordenadas del pie.
        xs, ys = (extraer_xy_de_video(vd[0]) if vd else extraer_xy_de_frames(fr))
        if heuristica_invalido(xs, ys):
            resultados.append({"grupo": car, "p_inv": p, "invalido": True,  #se agrega un registro de listado
                               "distancia_m": None, "calificacion": "Invalido"})#"invalido" es True "distancia_m": None porque no se mide distancia en inválidos y "calificacion": "Invalido".
            print(f"{nom}: Invalido (p_inv={p_txt}, heurística)") #imprime algo asi : nombredelsalto(depende como lo guardaste): Invalido (p_inv=0.55, heurística)
            continue
#si la heurística confirma que es inválido se acaba, ya no procesa ese grupo.
#si NO lo ve inválido, seguimos a medir distancia como válido (con margen dudoso)

    # Válido con margen => medir distancia
    #para que no te confndas con fs y fp te voy a explicar que es fr y vd.
    #fr: lista de rutas a imágenes (frames) dentro de la carpeta del grupo car.
    #vd: lista de videos encontrados en la misma carpeta. Usamos ambos porque el pipeline acepta desde frames sueltos hasta video por carpeta.
# en xs, ys: si hay video (vd no está vacío), usa el primero vd[0]. Si no, procesa la secuencia de frames fr.
# recordar que xs = posiciones horizontales (0 izquierda y 1 derecha). ys = posiciones verticales (0 arriba y 1 abajo).
#se calcula la distancia del salo con: el segmento del salto detectado (despegue, aterrizaje) si existe.
#Fallback: rango horizontal si no se pudo segmentar. ESCALA_METROS (13.2) convierte del rango normalizado (0–1) a metros reales.
#if vd y elif fr: Si hay video original escribe sobre el video (overlay_video).
#si hay solo frames se compone un video desde las imágenes (overlay_frames).
#nota importante: esto solo ocurre si GENERAR_OVERLAYS == True (las funciones mismas respetan ese switch).

    fr = listar_frames_en_carpeta(car); vd = listar_videos(car)
    xs, ys = (extraer_xy_de_video(vd[0]) if vd else extraer_xy_de_frames(fr)) #Extrae la trayectoria del pie (normalizadas) en el  tiempo.
    dist = calcular_distancia_solo_segmento(xs, ys, ESCALA_METROS)#calcula la distancia horizontal del salto en m
    nota = calificacion(dist) #como si traduciera la distancia en una etiqueta cualitativa.
    texto = "Sin coordenadas" if dist is None else f"{dist:.2f} m • {nota} (p_inv={p_txt})" #muestra metros con 2 decimales, la nota, y la probabilidad p_inv estimada por el modelo (p_txt, ya formateada antes) y si no hay dice que no hubo.
    if vd: overlay_video(vd[0], texto, (0,255,0), out_name) #genera el video de salida con overlay en verde (0,255,0):
    elif fr: overlay_frames(fr, texto, (0,255,0), out_name)

#Guarda el registro para el CSV: grupo: ruta de la carpeta del salto.
#p_inv: probabilidad de inválido (float o None).
#invalido: False (estamos en rama de válido).
#distancia_m: None si no hubo coordenadas; si sí, la distancia redondeada a 2 decimales.
#calificacion: “No detectado” o la nota (“normalito”, “muy bien” y asi).

    resultados.append({"grupo": car, "p_inv": p, "invalido": False,
                       "distancia_m": None if dist is None else round(dist, 2),
                       "calificacion": "No detectado" if dist is None else nota})

    if dist is None: #logs en consola.
        print(f"{nom}: Sin coordenadas (p_inv={p_txt})") #sin coordenadas:te avisa para revisar ese grupo (quizá MediaPipe falló).
    else:
        print(f"{nom}: {dist:.2f} m • {nota} (p_inv={p_txt})") #con distancia:imprime la métrica, calificación y probabilidad del modelo.

#resumen:extrae la trayectoria del pie con MediaPipe (video o frames).
#mide la distancia del salto (en metros) con el escale factor.
#etiqueta con una calificación textual y hace overlay en verde si está habilitado.
#Registra el resultado en la lista resultados y loggea.

# CSV
#es el cierre de todo el pipeline, se convierten los resultados en un CSV
#df esun dataframe de pandas y la fuente es la lista "resultados" que se fue llenando durante el loop principal.
#cada elemento de "resultados" es un diccionario que tiene: "grupo":carpeta o ruta del salto.
#"p_inv":probabilidad estimada de inválido (float o None).
#"invalido":True si es inválido, False si es válido.
#"distancia_m":distancia medida en metros (None si no hubo coordenadas) y "calificacion": que es la etiqueta textual ("Invalido", "Muy bueno", etc.).
#index=False evita que se guarde la columna numérica extra de índice (0,1,2,...) queda un archivo normal, que se puede abrir en Excel y cualquier editor de texto.
#el ultimo print mide y muestra el tiempo total de ejecución desde que guarde t0 = time.time().
#time.time()-t0 son los segundos transcurridos, formateados con 1 decimal.
#BATCH={N_BATCH} es el tamaño de lote se uso para clasificar (yo 16, cambialo si quieres, ya te explique ventajas y desventajas antes).
#OVERLAYS=on/off si se estaban generando videos con overlays o no (GENERAR_OVERLAYS).

df = pd.DataFrame(resultados, columns=["grupo","p_inv","invalido","distancia_m","calificacion"]) #columns es para que salgan en ese orden.
df.to_csv(RUTA_CSV, index=False) #Guarda el dataframe en un archivo CSV (en tu disco o drive en mi caso es drive)
print(f"\n[OK] CSV: {RUTA_CSV}")#mensaje de confirmación en consola. (para estar seguros).
print(f"[TIME] Total: {time.time()-t0:.1f}s  | BATCH={N_BATCH} | OVERLAYS={'on' if GENERAR_OVERLAYS else 'off'}")

# (Opcional) Métricas rápidas si el path revela la clase:
#por mucho que te diga que es opcional lo recomiendo ampliamente porque este sirve como evaluación rápida del modelo o sea, no recorre todo TensorFlow, solo mira el CSV de resultados.
#if len(df)solo se ejecuta si el DataFrame df no está vacío.Esto depende de que la ruta del grupo ("grupo") tenga en su nombre si es válido o no válido (o sea, /videosnovalidos/ en mi caso, pero tu mira como se llaman las rutas de tus videos).
#df["y"] crea una columna nueva "y" con la etiqueta real de la clase: Convierte grupo a string. Marca 1 si el path contiene /videosnovalidos/.
#Marca 0 si no lo contiene (válido). Esto es como un hack porque se usa la carpeta de origen como un ground truth.(o sea frames ya verificados por alguien, se sabe que son correctos)
#df["p"] crea la columna "p" con las probabilidades predichas. Si p_inv es NaN (no se pudo procesar), se rellena con -1.
#el prec, rec, f1 son metricas clasicas de clasificación binaria (Precisión, Recall (sensiblidad), F1 Score)
#la precisión contesta de todos los predichos como inválidos, ¿cuántos eran realmente inválidos? "1e-9" para evitar dividir por 0.
#el recall es V que detecto como NV. De los casos no válidos, ¿cuántos encontró el modelo?
#f1 score es una medida de equilibrio entre presición y recall o sea si la precisión es alta pero el recall bajo (o viceversa), el F1 score refleja ese desbalance. Toma valores entre 0 y 1, donde 1 es perfecto balance.
#precisión alta y recall bajo:el modelo solo marca como NV cuando está muy seguro, pero deja escapar muchos NV.
#recall alto y precisión baja:el modelo detecta casi todos los NV, pero también marca muchos V como NV.
#f1 score: mide qué tan buen balance se logra entre ambas situaciones. (se les llama "matriz de confución" a los tp, fpl tn, fn)
#resumen: este bloque es una validación express, qué tan bien trabaja el modelo al clasificar “inválidos” vs “válidos”
#es rapido porque esta usando la carpeta de origen como etiqueta real(ground truth).

if len(df):
    df["y"] = df["grupo"].astype(str).str.contains("/videosnovalidos/").astype(int)
    df["p"] = df["p_inv"].fillna(-1) #calcula las predicciones binarias (1 noválido, 0 válido)
    pr = (df["p"] >= UMBRAL_INVALIDO).astype(int)
    tp = int(((pr==1)&(df.y==1)).sum()) #tp (True Positive): predijo inválido y era inválido.
    fp = int(((pr==1)&(df.y==0)).sum()) #fp (False Positive): predijo inválido pero era válido.
    tn = int(((pr==0)&(df.y==0)).sum()) #tn (True Negative): predijo válido y era válido.
    fn = int(((pr==0)&(df.y==1)).sum()) #fn (False Negative): predijo válido pero era inválido.
    prec = tp/(tp+fp+1e-9); rec = tp/(tp+fn+1e-9); f1 = 2*prec*rec/(prec+rec+1e-9)
    print(f"\n[MÉTRICAS] (clase=Inválido)\nTP={tp} FP={fp} TN={tn} FN={fn}\nPrecisión={prec:.3f}  Recall={rec:.3f}  F1={f1:.3f}") #muestra la matriz de confusión y las métricas en 3 decimales.


[INFO] grupos totales: 102 (V=83, NV=19)
vn1: Invalido (p_inv=0.70)
vn2: 6.56 m • muy bien (p_inv=0.28)
vn3: 8.24 m • Muy Bueno (p_inv=0.44)
vn4: 10.52 m • perfecto (p_inv=0.39)
vn5: 7.47 m • muy bien (p_inv=0.16)
vn6: Invalido (p_inv=0.81)
vn7: 6.87 m • muy bien (p_inv=0.35)
vn8: Invalido (p_inv=0.82)
vn9: Invalido (p_inv=0.79)
vn10: 5.65 m • normalito (p_inv=0.47)
vn11: Invalido (p_inv=0.90)
vn12: 5.90 m • normalito (p_inv=0.48)
vn13: Invalido (p_inv=0.74)
vn14: Invalido (p_inv=0.78)
vn15: Invalido (p_inv=0.86)
vn16: Invalido (p_inv=0.85)
vn17: Invalido (p_inv=0.83)
vn18: Revisar (p_inv=0.55)
vn19: Invalido (p_inv=0.81)
Men's Long Jump Final _ Tokyo Replays - YouTube - Google Chrome 2025-01-03 15-48-46: 9.54 m • perfecto (p_inv=0.43)
Men's Long Jump Final _ Tokyo Replays - YouTube - Google Chrome 2025-01-03 15-48-46_result: 9.57 m • perfecto (p_inv=0.39)
Men's Long Jump Final _ Tokyo Replays - YouTube - Google Chrome 2025-01-03 15-49-07: 9.83 m • perfecto (p_inv=0.33)
Men's Long Jump

In [ ]:
import os, re, numpy as np, pandas as pd
#os para archivos, re para limpiar y parsear datos, np operaciones numéricas y pd tablas y leer CSV

# --- rutas y umbral (usa el de la sesión si existe) ---
RUTA = "/content/drive/My Drive/reporte_saltos.csv" #define la ruta del CSV que se generó en la inferencia.
try:
    UMBRAL_INVALIDO #intenta usar este valor (ya existe desde inferencia)
except NameError: #si no existe da el name error
    UMBRAL_INVALIDO = 0.60  # lo define en .60 si no existe.
#esta celda es como portable o sea que funciona con o sin tener la celda previa ejecutada.

if not os.path.exists(RUTA): #comprueba si el csv existe y si sí lo carga a df.
    print("No existe el CSV:", RUTA) #si no imprime un aviso y no intenta leer.
else:
    df = pd.read_csv(RUTA) #Si existe, lo lee en un DataFrame df.
#de aquí en adelante, df contendrá las columnas que genere en el pipeline(grupo, p_inv, invalido, distancia_m, calificacion) y las que luego agregue o tú lector).

    # --- limpieza de tipos ---
#p_inv y distancia_m se convierten a floats (NaN si no se puede). invalido se normaliza a boolean (True/False).
#para estar seguro de que el dataframe df este en un estado coherente para el análisis posterior (métricas, conteos, comparaciones)

    if "p_inv" in df.columns: #devuelve la lista de nombres de columnas en el dataframe y asegurar la columna p_inv sea de tipo numérico (float).
        df["p_inv"] = pd.to_numeric(df["p_inv"], errors="coerce") #eso con pd... y errors... las cosas no convertibles(texto, NaN, - cosas asi) son NaN. (Not a Number, el valor nulo numérico en pandas).
    if "distancia_m" in df.columns: #hace lo mismo pero para la columna de distancia en metros.
        df["distancia_m"] = pd.to_numeric(df["distancia_m"], errors="coerce")
    if "invalido" in df.columns: #asegura que la columna "inválido" exista
        df["invalido"] = (df["invalido"].astype(str).str.lower() #pasa los valores a strings y a minúsculas
                          .map({"true":True, "false":False, "1":True, "0":False})) #map pasa los strings a boleanos (T o F) si sale algo distinto pasa a NaN.

    # --- inferir ground truth (0 válido / 1 no válido) desde la ruta del grupo ---
#la ultima linea de este bloque aplica la función infer_gt sobre la columna "grupo" del DataFrame df.
#Resultado: crea una nueva columna "gt" (ground truth).
#Si la columna "grupo" no existe, simplemente rellena "gt" con NaN.

    def infer_gt(path):#es un ground truth en toda regla, se fija de donde salen los videos y asi los califica para no tener que hacer todo de nuevo.
        s = str(path).lower() #pasa a minúsculas.
        if "videosnovalidos" in s: return 1 #si la ruta esta en la carpeta videosnovalidos, el salto es inválido o sea etiqueta 1.
        if "videosvalidos"  in s: return 0 #lo mismo pero para válidos y etiqueta 0.
        return np.nan #si no se encuentra ninguna de esas palabras, no se puede deducir (NaN).

#aplica la función infer_gt sobre la columna "grupo" del DataFrame df, eso crea una nueva columna "gt" (ground truth).
#Si la columna "grupo" no existe, rellena "gt" con NaN.

    df["gt"] = df["grupo"].apply(infer_gt) if "grupo" in df.columns else np.nan


    # --- impresión básica ---
    print("HEAD(10):")
    print(df.head(10)) #devuelve las primeras 10 filas del DataFrame df.
    if "calificacion" in df.columns: #verifica si la columna "calificacion" existe en el DataFrame.
        print("\nConteo por calificación:\n", df["calificacion"].value_counts(dropna=False)) #dropna=False para que también se cuenten los valores nulos (NaN).

    # --- métricas si hay ground truth disponible ---
#df...any() devuelve True si hay al menos un valor no nulo en esa columna.
#has_gt será True solo si la columna gt existe y tiene algún valor válido (no todos son NaN).
#las metricas solo se van a calcular si hay ground truth (gt) y si existe la columna invalido (predicciones del modelo).
#sub=... crea un DataFrame sub eliminando cualquier fila donde gt o invalido sea NaN.
#para asegura que solo trabajamos con filas con comparación válida. .copy() garantiza que el DataFrame es separado y advertencias de SettingWithCopy en pandas.
#calcula la matriz de confusión (TP, FP, TN, FN) y las métricas estándar de clasificación (Precisión, Recall, F1).
#para evaluar qué tan bien el sistema detecta saltos inválidos frente al ground truth.

    has_gt = df["gt"].notna().any() if "gt" in df.columns else False #se revisa si en df hay una columna gt (ground truth)
    if has_gt and "invalido" in df.columns:
        sub = df.dropna(subset=["gt", "invalido"]).copy() #dropna para haceer un subconjunto.copy(),sub independiente de df para modificar gt con lo de abajo.
        sub["gt"] = sub["gt"].astype(int) #convierte la columna gt a enteros (recordar que 0 para válido, 1 para inválido).
        tp = int(((sub["invalido"]==True)  & (sub["gt"]==1)).sum()) #True Positive(tp):casos donde el modelo predijo invalido=True y el ground truth también era 1 (inválido).
        fp = int(((sub["invalido"]==True)  & (sub["gt"]==0)).sum()) #False Positive(fp):el modelo dijo inválido(True)pero el gt era válido (0).
        tn = int(((sub["invalido"]==False) & (sub["gt"]==0)).sum()) #True Negative(tn):el modelo dijo válido (False) y el gt era válido (0).
        fn = int(((sub["invalido"]==False) & (sub["gt"]==1)).sum()) #False Negative(fn):el modelo dijo válido (False), pero en realidad era inválido (1)
        prec = tp / (tp + fp + 1e-9) #Precisión:de todos los que el modelo marcó como inválidos, ¿qué porcentaje realmente eran inválidos? es lo mismo de la última vez.
        rec  = tp / (tp + fn + 1e-9) #Recall (Sensibilidad o Tasa Verdaderos Positivos):de todos los inválidos reales, ¿cuántos detectó el modelo?
        f1   = 2*prec*rec/(prec+rec + 1e-9) #F1-score:media entre precisión y recall. Sirve como balance cuando quieres considerar ambas métricas.
        print("\n[METRICAS] (clase=Inválido)") #imprime la matriz de confusión y las métricas redondeadas a 3 decimales.
        print(f"TP={tp} FP={fp} TN={tn} FN={fn}")
        print(f"Precision={prec:.3f}  Recall={rec:.3f}  F1={f1:.3f}")

    # --- casos más dudosos (p_inv cerca del umbral) ---
#se crea una nueva columna _delta(auxiliar más que privada)
#. _delta = distancia absoluta entre la probabilidad p_inv y el umbral.
#si _delta es pequeña, significa que el modelo estuvo muy cerca de la frontera entre válido/inválido y eso es un caso dudoso.
#en dudosos se elimina filas donde p_inv es NaN, ordena el DataFrame de menor a mayor en _delta, primero aparecerán los más ambiguos.
#head(12) para que se muestren los 12 casos más cercanos al umbral.
#cols_show incluye solo los que realmente existan en el DataFrame:grupo es nombre o carpeta del salto.
#p_inv es la probabilidad de inválido. invalido es la predicción del sistema (True/False).
#gt es el ground truth (si existe). distancia_m es la distancia estimada del salto. calificacion es texto con nota.
#_delta es qué tan lejos del umbral quedó. c es la variable de iteración, la primera es el ekemento que se va agregando y la segunda es condición, solo incluye c si la columna existe.
#Este bloque es para revisar los saltos más ambiguos o sea aquellos donde el modelo estuvo indeciso entre válido/inválido.
#es útil para auditoría (revisar casos dudosos), debugging (desarrollar o ajusta el sistema pero para ver si tiene lógica) y decidir si ajustar el umbral.

    if "p_inv" in df.columns: #verifica que existe la columna p_inv (probabilidad de inválido). Si no existe, el bloque no se ejecuta.
        df["_delta"] = (df["p_inv"] - float(UMBRAL_INVALIDO)).abs()
        dudosos = df.dropna(subset=["p_inv"]).sort_values("_delta").head(12)
        cols_show = [c for c in ["grupo","p_inv","invalido","gt","distancia_m","calificacion","_delta"] if c in dudosos.columns]
        print(f"\n[CASOS DUDOSOS] (más cerca del umbral={UMBRAL_INVALIDO:.2f})") #print a 2 decimales.
        print(dudosos[cols_show].to_string(index=False)) #agarra del dataframe "dudosos" solo las columnas definidas en cols_show.
#convierte ese dataframe en una cadena bonita para imprimir, sin índice.

    # --- stats de distancias (si existen) ---
#count es el número de distancias válidas. mean es la media (promedio). std es la desviación estándar.dan una idea rápida del tamaño de la muestra
#el promedio de saltos y qué tanto varían.


    if "distancia_m" in df.columns: #se asegura que el DataFrame tenga una columna llamada distancia_m.es donde guarde la distancia en metros.
        dist_ok = df["distancia_m"].dropna() #crea una serie dist_ok que tiene distancias válidas (ignora saltos donde no se pudo calcular nada).
        if not dist_ok.empty: #antes de calcular estadísticas, se asegura de que haya al menos un valor válido.
            q = dist_ok.quantile([0,0.25,0.5,0.75,1.0]).round(2) #calcula cuantiles (límite estadístico)
            print("\n[STATS distancia_m] count={}, mean={:.2f}, std={:.2f}".format(len(dist_ok), dist_ok.mean(), dist_ok.std()))
            print("Quantiles:\n", q) #muestra la tabla con los cuantiles para que veas cómo se distribuyen los saltos.

    # (Opcional) exportar a Excel para compartir
    # df.to_excel("/content/drive/My Drive/reporte_saltos.xlsx", index=False)
    #seria para guardar en mi drive porque estoy en colab


HEAD(10):
                                               grupo     p_inv  invalido  \
0  /content/drive/My Drive/videoscombi/videosnova...  0.695404      True   
1  /content/drive/My Drive/videoscombi/videosnova...  0.275340     False   
2  /content/drive/My Drive/videoscombi/videosnova...  0.441275     False   
3  /content/drive/My Drive/videoscombi/videosnova...  0.393677     False   
4  /content/drive/My Drive/videoscombi/videosnova...  0.155577     False   
5  /content/drive/My Drive/videoscombi/videosnova...  0.810554      True   
6  /content/drive/My Drive/videoscombi/videosnova...  0.347080     False   
7  /content/drive/My Drive/videoscombi/videosnova...  0.817027      True   
8  /content/drive/My Drive/videoscombi/videosnova...  0.794558      True   
9  /content/drive/My Drive/videoscombi/videosnova...  0.465545     False   

   distancia_m calificacion  gt  
0          NaN     Invalido   1  
1         6.56     muy bien   1  
2         8.24    Muy Bueno   1  
3        10.52   